# Visualize pymoo result
Load the pymoo result object saved with pickle and visualize it.

ONOSControllerPlacement needs to be definded to load the result object.
So, execute the following cell first.

In [1]:
#!/usr/bin/env python
import numpy as np
import networkx as nx
import math
import pickle
from pymoo.core.problem import ElementwiseProblem

class ONOSControllerPlacement(ElementwiseProblem):
    def __init__(self, num_nodes, distance_matrix, shortest_paths, graph, **kwargs):
        super().__init__(n_var=2*num_nodes, 
                         n_obj=4, 
                         n_constr=2, 
                         xl=0, xu=1, 
                         **kwargs)
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix
        self.shortest_paths = shortest_paths
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        controller_nodes = x[:self.num_nodes]   # first half is controller placement
        atomix_nodes = x[self.num_nodes:]       # second half is atomix placement


        num_controller = np.sum(controller_nodes)
        num_atomix = np.sum(atomix_nodes)

        # Obj1: Minimize number of contrtoller
        f1 = num_controller

        # Obj2: Minimize number of atomix
        f2 = num_atomix

        # Obj3: Minimize average FSP
        f3 = calculate_FST(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                           self.shortest_paths)
        
        f4 = calculate_BC(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                        #    self.shortest_paths,
                           self.graph)

        # Constr1: The number of controller is equal to or greater than 2
        g1 = 2 - num_controller

        # Constr2: The number of atomix is equal to or greater than 3
        g2 = 3 - num_atomix
        
        # Add the centrality metrix into optimazing objectives:
        # 1. Nearest controller for each switch
        # 2. The number of controlled switches for each controller should be <= limit_num_switches_controlled (limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)))
        # 3. return value should be the variance for all controller's betweenness centrality
        out["F"] = [f1, f2, f3, f4]
        out["G"] = [g1, g2]


def calculate_FST(num_nodes, controller_nodes, atomix_nodes, distance_matrix, shortest_paths):
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()
    atomix_list = np.nonzero(atomix_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        for c in controller_list:
            if distance_matrix[s][c] < delay:
                delay = distance_matrix[s][c]
                nearest_controller = c
        controller_of.append(nearest_controller)    

    # calculate average delay to atomix nodes from each controller
    average_atomix_delay_from = {}
    for c in controller_list:
        delay = []
        for a in atomix_list:
            delay.append(distance_matrix[c][a])
        average_atomix_delay_from[c] = np.mean(delay)

    # find the nearest atomix for each atomix and calculate average delay
    atomix_atomix_delays = []
    for a1 in atomix_list:
        delay = math.inf
        for a2 in atomix_list:
            if(a1 == a2):
                continue
            if distance_matrix[a1][a2] < delay:
                delay = distance_matrix[a1][a2]
        atomix_atomix_delays.append(delay)
    average_atomix_atomix_delay = np.mean(atomix_atomix_delays)
    FTSs = []
    for source in range(num_nodes):
        for distination in range(num_nodes):
            if(source == distination):
                continue
            delay = 0
            is_controlled_by_single_controller = True
            counted_controllers = []
            for s in shortest_paths[source][distination]:
                # switch-controller delay
                delay += distance_matrix[s][controller_of[s]] * 4

                # controller-atomix delay
                if(s == source):
                    delay += average_atomix_delay_from[controller_of[s]] * 2
                elif(s != distination):
                    if(controller_of[s] != controller_of[source]):
                        is_controlled_by_single_controller = False
                        if(not controller_of[s] in counted_controllers):
                            counted_controllers.append(controller_of[s])
                            delay += average_atomix_delay_from[controller_of[s]]
                else:
                    if(controller_of[s] == controller_of[source]):
                        if(not is_controlled_by_single_controller):
                            delay += average_atomix_delay_from[controller_of[s]]
                    else:
                        delay += average_atomix_delay_from[controller_of[s]] * 2
            
            # atomix-atomix delay
            delay +=  average_atomix_atomix_delay * 2
            FTSs.append(delay)

    return np.mean(FTSs)



def calculate_BC(num_nodes, controller_nodes, atomix_nodes, distance_matrix, graph):
    G = nx.Graph()
    for node1 in range(len(graph)):
        G.add_node(str(node1))
        for node2, delay in graph[node1].items():
            G.add_edge(str(node1), str(node2), weight=delay)
    
    # The list of betweenness centrality for all switches
    nodes_bc=nx.current_flow_betweenness_centrality(G, normalized=True, weight=None, dtype='float', solver='full')
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)) # balance the number of switches controllers can control 
    switches_bc_of_controller_ = dict.fromkeys((range(num_nodes)),0) # list of sum of betweenness centrality of switches for each controller
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        controlled_switches=[]
        for c in controller_list:
            # Conditions: nearest controller (with the lowest delay) && the number of switches for each controller < limit_num_switches_controlled
            if distance_matrix[s][c] < delay and controller_of.count(c) < limit_num_switches_controlled:
                delay = distance_matrix[s][c]
                nearest_controller = c
                controlled_switches.append(s)
        switches_bc_of_controller_[nearest_controller] += nodes_bc[str(s)]
        controller_of.append(nearest_controller)
    
    # Simplify switches_bc_of_controller_ (only need value for calculating variance)
    bc_array = []
    for i in switches_bc_of_controller_.values():
        bc_array.append(i)

    # return variance value can show the degree of balance within all controllers
    return np.var(bc_array)

## Load all results

### Load Cogent results for all algos

In [2]:
with open('res_bc_Cogent_agemoea.pkl','rb') as f1_Cogent:
    res_1_Cogent = pickle.load(f1_Cogent)
with open('res_bc_Cogent_agemoea2.pkl','rb') as f2_Cogent:
    res_2_Cogent = pickle.load(f2_Cogent)
with open('res_bc_Cogent_nsga2.pkl','rb') as f3_Cogent:
    res_3_Cogent = pickle.load(f3_Cogent)
with open('res_bc_Cogent_nsga3.pkl','rb') as f4_Cogent:
    res_4_Cogent = pickle.load(f4_Cogent)
with open('res_bc_Cogent_rnsga2.pkl','rb') as f5_Cogent:
    res_5_Cogent = pickle.load(f5_Cogent)
with open('res_bc_Cogent_rnsga3.pkl','rb') as f6_Cogent:
    res_6_Cogent = pickle.load(f6_Cogent)
with open('res_bc_Cogent_rvea.pkl','rb') as f7_Cogent:
    res_7_Cogent = pickle.load(f7_Cogent)
with open('res_bc_Cogent_smsemoa.pkl','rb') as f8_Cogent:
    res_8_Cogent = pickle.load(f8_Cogent)
with open('res_bc_Cogent_unsga3.pkl','rb') as f9_Cogent:
    res_9_Cogent = pickle.load(f9_Cogent)
with open('res_bc_Cogent_ctaea.pkl','rb') as f10_Cogent:
    res_10_Cogent = pickle.load(f10_Cogent)
with open('res_bc_Cogent_dnsga2.pkl','rb') as f11_Cogent:
    res_11_Cogent = pickle.load(f11_Cogent)
with open('res_bc_Cogent_kgbdmoea.pkl','rb') as f12_Cogent:
    res_12_Cogent = pickle.load(f12_Cogent)
with open('res_bc_Cogent_moead.pkl','rb') as f13_Cogent:
    res_13_Cogent = pickle.load(f13_Cogent)

### Load UsCarrier results for all algos

In [3]:
with open('res_bc_UsCarrier_agemoea.pkl','rb') as f1_UsCarrier:
    res_1_UsCarrier = pickle.load(f1_UsCarrier)
with open('res_bc_UsCarrier_agemoea2.pkl','rb') as f2_UsCarrier:
    res_2_UsCarrier = pickle.load(f2_UsCarrier)
with open('res_bc_UsCarrier_nsga2.pkl','rb') as f3_UsCarrier:
    res_3_UsCarrier = pickle.load(f3_UsCarrier)
with open('res_bc_UsCarrier_nsga3.pkl','rb') as f4_UsCarrier:
    res_4_UsCarrier = pickle.load(f4_UsCarrier)
with open('res_bc_UsCarrier_rnsga2.pkl','rb') as f5_UsCarrier:
    res_5_UsCarrier = pickle.load(f5_UsCarrier)
with open('res_bc_UsCarrier_rnsga3.pkl','rb') as f6_UsCarrier:
    res_6_UsCarrier = pickle.load(f6_UsCarrier)
with open('res_bc_UsCarrier_rvea.pkl','rb') as f7_UsCarrier:
    res_7_UsCarrier = pickle.load(f7_UsCarrier)
with open('res_bc_UsCarrier_smsemoa.pkl','rb') as f8_UsCarrier:
    res_8_UsCarrier = pickle.load(f8_UsCarrier)
with open('res_bc_UsCarrier_unsga3.pkl','rb') as f9_UsCarrier:
    res_9_UsCarrier = pickle.load(f9_UsCarrier)
with open('res_bc_UsCarrier_ctaea.pkl','rb') as f10_UsCarrier:
    res_10_UsCarrier = pickle.load(f10_UsCarrier)
with open('res_bc_UsCarrier_dnsga2.pkl','rb') as f11_UsCarrier:
    res_11_UsCarrier = pickle.load(f11_UsCarrier)
with open('res_bc_UsCarrier_kgbdmoea.pkl','rb') as f12_UsCarrier:
    res_12_UsCarrier = pickle.load(f12_UsCarrier)
with open('res_bc_UsCarrier_moead.pkl','rb') as f13_UsCarrier:
    res_13_UsCarrier = pickle.load(f13_UsCarrier)

: 

### Load HiberniaGlobal results for all algos

In [ ]:
with open('res_bc_HiberniaGlobal_agemoea.pkl','rb') as f1_HiberniaGlobal:
    res_1_HiberniaGlobal = pickle.load(f1_HiberniaGlobal)
with open('res_bc_HiberniaGlobal_agemoea2.pkl','rb') as f2_HiberniaGlobal:
    res_2_HiberniaGlobal = pickle.load(f2_HiberniaGlobal)
with open('res_bc_HiberniaGlobal_nsga2.pkl','rb') as f3_HiberniaGlobal:
    res_3_HiberniaGlobal = pickle.load(f3_HiberniaGlobal)
with open('res_bc_HiberniaGlobal_nsga3.pkl','rb') as f4_HiberniaGlobal:
    res_4_HiberniaGlobal = pickle.load(f4_HiberniaGlobal)
with open('res_bc_HiberniaGlobal_rnsga2.pkl','rb') as f5_HiberniaGlobal:
    res_5_HiberniaGlobal = pickle.load(f5_HiberniaGlobal)
with open('res_bc_HiberniaGlobal_rnsga3.pkl','rb') as f6_HiberniaGlobal:
    res_6_HiberniaGlobal = pickle.load(f6_HiberniaGlobal)
with open('res_bc_HiberniaGlobal_rvea.pkl','rb') as f7_HiberniaGlobal:
    res_7_HiberniaGlobal = pickle.load(f7_HiberniaGlobal)
with open('res_bc_HiberniaGlobal_smsemoa.pkl','rb') as f8_HiberniaGlobal:
    res_8_HiberniaGlobal = pickle.load(f8_HiberniaGlobal)
with open('res_bc_HiberniaGlobal_unsga3.pkl','rb') as f9_HiberniaGlobal:
    res_9_HiberniaGlobal = pickle.load(f9_HiberniaGlobal)
with open('res_bc_HiberniaGlobal_ctaea.pkl','rb') as f10_HiberniaGlobal:
    res_10_HiberniaGlobal = pickle.load(f10_HiberniaGlobal)
with open('res_bc_HiberniaGlobal_dnsga2.pkl','rb') as f11_HiberniaGlobal:
    res_11_HiberniaGlobal = pickle.load(f11_HiberniaGlobal)
with open('res_bc_HiberniaGlobal_kgbdmoea.pkl','rb') as f12_HiberniaGlobal:
    res_12_HiberniaGlobal = pickle.load(f12_HiberniaGlobal)
with open('res_bc_HiberniaGlobal_moead.pkl','rb') as f13_HiberniaGlobal:
    res_13_HiberniaGlobal = pickle.load(f13_HiberniaGlobal)

### Load Colt results for all algos

In [ ]:
with open('res_bc_Colt_agemoea.pkl','rb') as f1_Colt:
    res_1_Colt = pickle.load(f1_Colt)
with open('res_bc_Colt_agemoea2.pkl','rb') as f2_Colt:
    res_2_Colt = pickle.load(f2_Colt)
with open('res_bc_Colt_nsga2.pkl','rb') as f3_Colt:
    res_3_Colt = pickle.load(f3_Colt)
with open('res_bc_Colt_nsga3.pkl','rb') as f4_Colt:
    res_4_Colt = pickle.load(f4_Colt)
with open('res_bc_Colt_rnsga2.pkl','rb') as f5_Colt:
    res_5_Colt = pickle.load(f5_Colt)
with open('res_bc_Colt_rnsga3.pkl','rb') as f6_Colt:
    res_6_Colt = pickle.load(f6_Colt)
with open('res_bc_Colt_rvea.pkl','rb') as f7_Colt:
    res_7_Colt = pickle.load(f7_Colt)
with open('res_bc_Colt_smsemoa.pkl','rb') as f8_Colt:
    res_8_Colt = pickle.load(f8_Colt)
with open('res_bc_Colt_unsga3.pkl','rb') as f9_Colt:
    res_9_Colt = pickle.load(f9_Colt)
with open('res_bc_Colt_ctaea.pkl','rb') as f10_Colt:
    res_10_Colt = pickle.load(f10_Colt)
with open('res_bc_Colt_dnsga2.pkl','rb') as f11_Colt:
    res_11_Colt = pickle.load(f11_Colt)
with open('res_bc_Colt_kgbdmoea.pkl','rb') as f12_Colt:
    res_12_Colt = pickle.load(f12_Colt)
with open('res_bc_Colt_moead.pkl','rb') as f13_Colt:
    res_13_Colt = pickle.load(f13_Colt)

### Load Funet results for all algos

In [ ]:
with open('res_bc_Funet_agemoea.pkl','rb') as f1_Funet:
    res_1_Funet = pickle.load(f1_Funet)
with open('res_bc_Funet_agemoea2.pkl','rb') as f2_Funet:
    res_2_Funet = pickle.load(f2_Funet)
with open('res_bc_Funet_nsga2.pkl','rb') as f3_Funet:
    res_3_Funet = pickle.load(f3_Funet)
with open('res_bc_Funet_nsga3.pkl','rb') as f4_Funet:
    res_4_Funet = pickle.load(f4_Funet)
with open('res_bc_Funet_rnsga2.pkl','rb') as f5_Funet:
    res_5_Funet = pickle.load(f5_Funet)
with open('res_bc_Funet_rnsga3.pkl','rb') as f6_Funet:
    res_6_Funet = pickle.load(f6_Funet)
with open('res_bc_Funet_rvea.pkl','rb') as f7_Funet:
    res_7_Funet = pickle.load(f7_Funet)
with open('res_bc_Funet_smsemoa.pkl','rb') as f8_Funet:
    res_8_Funet = pickle.load(f8_Funet)
with open('res_bc_Funet_unsga3.pkl','rb') as f9_Funet:
    res_9_Funet = pickle.load(f9_Funet)
with open('res_bc_Funet_ctaea.pkl','rb') as f10_Funet:
    res_10_Funet = pickle.load(f10_Funet)
with open('res_bc_Funet_dnsga2.pkl','rb') as f11_Funet:
    res_11_Funet = pickle.load(f11_Funet)
with open('res_bc_Funet_kgbdmoea.pkl','rb') as f12_Funet:
    res_12_Funet = pickle.load(f12_Funet)
with open('res_bc_Funet_moead.pkl','rb') as f13_Funet:
    res_13_Funet = pickle.load(f13_Funet)

### Load Abvt results for all algos

In [ ]:
with open('res_bc_Abvt_agemoea.pkl','rb') as f1_Abvt:
    res_1_Abvt = pickle.load(f1_Abvt)
with open('res_bc_Abvt_agemoea2.pkl','rb') as f2_Abvt:
    res_2_Abvt = pickle.load(f2_Abvt)
with open('res_bc_Abvt_nsga2.pkl','rb') as f3_Abvt:
    res_3_Abvt = pickle.load(f3_Abvt)
with open('res_bc_Abvt_nsga3.pkl','rb') as f4_Abvt:
    res_4_Abvt = pickle.load(f4_Abvt)
with open('res_bc_Abvt_rnsga2.pkl','rb') as f5_Abvt:
    res_5_Abvt = pickle.load(f5_Abvt)
with open('res_bc_Abvt_rnsga3.pkl','rb') as f6_Abvt:
    res_6_Abvt = pickle.load(f6_Abvt)
with open('res_bc_Abvt_rvea.pkl','rb') as f7_Abvt:
    res_7_Abvt = pickle.load(f7_Abvt)
with open('res_bc_Abvt_smsemoa.pkl','rb') as f8_Abvt:
    res_8_Abvt = pickle.load(f8_Abvt)
with open('res_bc_Abvt_unsga3.pkl','rb') as f9_Abvt:
    res_9_Abvt = pickle.load(f9_Abvt)
with open('res_bc_Abvt_ctaea.pkl','rb') as f10_Abvt:
    res_10_Abvt = pickle.load(f10_Abvt)
with open('res_bc_Abvt_dnsga2.pkl','rb') as f11_Abvt:
    res_11_Abvt = pickle.load(f11_Abvt)
with open('res_bc_Abvt_kgbdmoea.pkl','rb') as f12_Abvt:
    res_12_Abvt = pickle.load(f12_Abvt)
with open('res_bc_Abvt_moead.pkl','rb') as f13_Abvt:
    res_13_Abvt = pickle.load(f13_Abvt)

### Load Intellifiber results for all algos

In [ ]:
with open('res_bc_Intellifiber_agemoea.pkl','rb') as f1_Intellifiber:
    res_1_Intellifiber = pickle.load(f1_Intellifiber)
with open('res_bc_Intellifiber_agemoea2.pkl','rb') as f2_Intellifiber:
    res_2_Intellifiber = pickle.load(f2_Intellifiber)
with open('res_bc_Intellifiber_nsga2.pkl','rb') as f3_Intellifiber:
    res_3_Intellifiber = pickle.load(f3_Intellifiber)
with open('res_bc_Intellifiber_nsga3.pkl','rb') as f4_Intellifiber:
    res_4_Intellifiber = pickle.load(f4_Intellifiber)
with open('res_bc_Intellifiber_rnsga2.pkl','rb') as f5_Intellifiber:
    res_5_Intellifiber = pickle.load(f5_Intellifiber)
with open('res_bc_Intellifiber_rnsga3.pkl','rb') as f6_Intellifiber:
    res_6_Intellifiber = pickle.load(f6_Intellifiber)
with open('res_bc_Intellifiber_rvea.pkl','rb') as f7_Intellifiber:
    res_7_Intellifiber = pickle.load(f7_Intellifiber)
with open('res_bc_Intellifiber_smsemoa.pkl','rb') as f8_Intellifiber:
    res_8_Intellifiber = pickle.load(f8_Intellifiber)
with open('res_bc_Intellifiber_unsga3.pkl','rb') as f9_Intellifiber:
    res_9_Intellifiber = pickle.load(f9_Intellifiber)
with open('res_bc_Intellifiber_ctaea.pkl','rb') as f10_Intellifiber:
    res_10_Intellifiber = pickle.load(f10_Intellifiber)
with open('res_bc_Intellifiber_dnsga2.pkl','rb') as f11_Intellifiber:
    res_11_Intellifiber = pickle.load(f11_Intellifiber)
with open('res_bc_Intellifiber_kgbdmoea.pkl','rb') as f12_Intellifiber:
    res_12_Intellifiber = pickle.load(f12_Intellifiber)
with open('res_bc_Intellifiber_moead.pkl','rb') as f13_Intellifiber:
    res_13_Intellifiber = pickle.load(f13_Intellifiber)

### Load TataNld results for all algos

In [ ]:
with open('res_bc_TataNld_agemoea.pkl','rb') as f1_TataNld:
    res_1_TataNld = pickle.load(f1_TataNld)
with open('res_bc_TataNld_agemoea2.pkl','rb') as f2_TataNld:
    res_2_TataNld = pickle.load(f2_TataNld)
with open('res_bc_TataNld_nsga2.pkl','rb') as f3_TataNld:
    res_3_TataNld = pickle.load(f3_TataNld)
with open('res_bc_TataNld_nsga3.pkl','rb') as f4_TataNld:
    res_4_TataNld = pickle.load(f4_TataNld)
with open('res_bc_TataNld_rnsga2.pkl','rb') as f5_TataNld:
    res_5_TataNld = pickle.load(f5_TataNld)
with open('res_bc_TataNld_rnsga3.pkl','rb') as f6_TataNld:
    res_6_TataNld = pickle.load(f6_TataNld)
with open('res_bc_TataNld_rvea.pkl','rb') as f7_TataNld:
    res_7_TataNld = pickle.load(f7_TataNld)
with open('res_bc_TataNld_smsemoa.pkl','rb') as f8_TataNld:
    res_8_TataNld = pickle.load(f8_TataNld)
with open('res_bc_TataNld_unsga3.pkl','rb') as f9_TataNld:
    res_9_TataNld = pickle.load(f9_TataNld)
with open('res_bc_TataNld_ctaea.pkl','rb') as f10_TataNld:
    res_10_TataNld = pickle.load(f10_TataNld)
with open('res_bc_TataNld_dnsga2.pkl','rb') as f11_TataNld:
    res_11_TataNld = pickle.load(f11_TataNld)
with open('res_bc_TataNld_kgbdmoea.pkl','rb') as f12_TataNld:
    res_12_TataNld = pickle.load(f12_TataNld)
with open('res_bc_TataNld_moead.pkl','rb') as f13_TataNld:
    res_13_TataNld = pickle.load(f13_TataNld)

### Load Internode results for all algos

In [ ]:
with open('res_bc_Internode_agemoea.pkl','rb') as f1_Internode:
    res_1_Internode = pickle.load(f1_Internode)
with open('res_bc_Internode_agemoea2.pkl','rb') as f2_Internode:
    res_2_Internode = pickle.load(f2_Internode)
with open('res_bc_Internode_nsga2.pkl','rb') as f3_Internode:
    res_3_Internode = pickle.load(f3_Internode)
with open('res_bc_Internode_nsga3.pkl','rb') as f4_Internode:
    res_4_Internode = pickle.load(f4_Internode)
with open('res_bc_Internode_rnsga2.pkl','rb') as f5_Internode:
    res_5_Internode = pickle.load(f5_Internode)
with open('res_bc_Internode_rnsga3.pkl','rb') as f6_Internode:
    res_6_Internode = pickle.load(f6_Internode)
with open('res_bc_Internode_rvea.pkl','rb') as f7_Internode:
    res_7_Internode = pickle.load(f7_Internode)
with open('res_bc_Internode_smsemoa.pkl','rb') as f8_Internode:
    res_8_Internode = pickle.load(f8_Internode)
with open('res_bc_Internode_unsga3.pkl','rb') as f9_Internode:
    res_9_Internode = pickle.load(f9_Internode)
with open('res_bc_Internode_ctaea.pkl','rb') as f10_Internode:
    res_10_Internode = pickle.load(f10_Internode)
with open('res_bc_Internode_dnsga2.pkl','rb') as f11_Internode:
    res_11_Internode = pickle.load(f11_Internode)
with open('res_bc_Internode_kgbdmoea.pkl','rb') as f12_Internode:
    res_12_Internode = pickle.load(f12_Internode)
with open('res_bc_Internode_moead.pkl','rb') as f13_Internode:
    res_13_Internode = pickle.load(f13_Internode)

### Load Missouri results for all algos

In [ ]:
with open('res_bc_Missouri_agemoea.pkl','rb') as f1_Missouri:
    res_1_Missouri = pickle.load(f1_Missouri)
with open('res_bc_Missouri_agemoea2.pkl','rb') as f2_Missouri:
    res_2_Missouri = pickle.load(f2_Missouri)
with open('res_bc_Missouri_nsga2.pkl','rb') as f3_Missouri:
    res_3_Missouri = pickle.load(f3_Missouri)
with open('res_bc_Missouri_nsga3.pkl','rb') as f4_Missouri:
    res_4_Missouri = pickle.load(f4_Missouri)
with open('res_bc_Missouri_rnsga2.pkl','rb') as f5_Missouri:
    res_5_Missouri = pickle.load(f5_Missouri)
with open('res_bc_Missouri_rnsga3.pkl','rb') as f6_Missouri:
    res_6_Missouri = pickle.load(f6_Missouri)
with open('res_bc_Missouri_rvea.pkl','rb') as f7_Missouri:
    res_7_Missouri = pickle.load(f7_Missouri)
with open('res_bc_Missouri_smsemoa.pkl','rb') as f8_Missouri:
    res_8_Missouri = pickle.load(f8_Missouri)
with open('res_bc_Missouri_unsga3.pkl','rb') as f9_Missouri:
    res_9_Missouri = pickle.load(f9_Missouri)
with open('res_bc_Missouri_ctaea.pkl','rb') as f10_Missouri:
    res_10_Missouri = pickle.load(f10_Missouri)
with open('res_bc_Missouri_dnsga2.pkl','rb') as f11_Missouri:
    res_11_Missouri = pickle.load(f11_Missouri)
with open('res_bc_Missouri_kgbdmoea.pkl','rb') as f12_Missouri:
    res_12_Missouri = pickle.load(f12_Missouri)
with open('res_bc_Missouri_moead.pkl','rb') as f13_Missouri:
    res_13_Missouri = pickle.load(f13_Missouri)

### Load Ion results for all algos

In [ ]:
with open('res_bc_Ion_agemoea.pkl','rb') as f1_Ion:
    res_1_Ion = pickle.load(f1_Ion)
with open('res_bc_Ion_agemoea2.pkl','rb') as f2_Ion:
    res_2_Ion = pickle.load(f2_Ion)
with open('res_bc_Ion_nsga2.pkl','rb') as f3_Ion:
    res_3_Ion = pickle.load(f3_Ion)
with open('res_bc_Ion_nsga3.pkl','rb') as f4_Ion:
    res_4_Ion = pickle.load(f4_Ion)
with open('res_bc_Ion_rnsga2.pkl','rb') as f5_Ion:
    res_5_Ion = pickle.load(f5_Ion)
with open('res_bc_Ion_rnsga3.pkl','rb') as f6_Ion:
    res_6_Ion = pickle.load(f6_Ion)
with open('res_bc_Ion_rvea.pkl','rb') as f7_Ion:
    res_7_Ion = pickle.load(f7_Ion)
with open('res_bc_Ion_smsemoa.pkl','rb') as f8_Ion:
    res_8_Ion = pickle.load(f8_Ion)
with open('res_bc_Ion_unsga3.pkl','rb') as f9_Ion:
    res_9_Ion = pickle.load(f9_Ion)
with open('res_bc_Ion_ctaea.pkl','rb') as f10_Ion:
    res_10_Ion = pickle.load(f10_Ion)
with open('res_bc_Ion_dnsga2.pkl','rb') as f11_Ion:
    res_11_Ion = pickle.load(f11_Ion)
with open('res_bc_Ion_kgbdmoea.pkl','rb') as f12_Ion:
    res_12_Ion = pickle.load(f12_Ion)
with open('res_bc_Ion_moead.pkl','rb') as f13_Ion:
    res_13_Ion = pickle.load(f13_Ion)

### Load Palmetto results for all algos

In [ ]:
with open('res_bc_Palmetto_agemoea.pkl','rb') as f1_Palmetto:
    res_1_Palmetto = pickle.load(f1_Palmetto)
with open('res_bc_Palmetto_agemoea2.pkl','rb') as f2_Palmetto:
    res_2_Palmetto = pickle.load(f2_Palmetto)
with open('res_bc_Palmetto_nsga2.pkl','rb') as f3_Palmetto:
    res_3_Palmetto = pickle.load(f3_Palmetto)
with open('res_bc_Palmetto_nsga3.pkl','rb') as f4_Palmetto:
    res_4_Palmetto = pickle.load(f4_Palmetto)
with open('res_bc_Palmetto_rnsga2.pkl','rb') as f5_Palmetto:
    res_5_Palmetto = pickle.load(f5_Palmetto)
with open('res_bc_Palmetto_rnsga3.pkl','rb') as f6_Palmetto:
    res_6_Palmetto = pickle.load(f6_Palmetto)
with open('res_bc_Palmetto_rvea.pkl','rb') as f7_Palmetto:
    res_7_Palmetto = pickle.load(f7_Palmetto)
with open('res_bc_Palmetto_smsemoa.pkl','rb') as f8_Palmetto:
    res_8_Palmetto = pickle.load(f8_Palmetto)
with open('res_bc_Palmetto_unsga3.pkl','rb') as f9_Palmetto:
    res_9_Palmetto = pickle.load(f9_Palmetto)
with open('res_bc_Palmetto_ctaea.pkl','rb') as f10_Palmetto:
    res_10_Palmetto = pickle.load(f10_Palmetto)
with open('res_bc_Palmetto_dnsga2.pkl','rb') as f11_Palmetto:
    res_11_Palmetto = pickle.load(f11_Palmetto)
with open('res_bc_Palmetto_kgbdmoea.pkl','rb') as f12_Palmetto:
    res_12_Palmetto = pickle.load(f12_Palmetto)
with open('res_bc_Palmetto_moead.pkl','rb') as f13_Palmetto:
    res_13_Palmetto = pickle.load(f13_Palmetto)

## Hypervolume
1. Store values separately

In [ ]:

# agemoea
F1_Cogent=res_1_Cogent.F
F1_UsCarrier=res_1_UsCarrier.F
F1_HiberniaGlobal=res_1_HiberniaGlobal.F
F1_Colt=res_1_Colt.F
F1_Funet=res_1_Funet.F
F1_Abvt=res_1_Abvt.F
F1_Intellifiber=res_1_Intellifiber.F
F1_TataNld=res_1_TataNld.F
F1_Internode=res_1_Internode.F
F1_Missouri=res_1_Missouri.F
F1_Ion=res_1_Ion.F
F1_Palmetto=res_1_Palmetto.F

# agemoea2
F2_Cogent=res_2_Cogent.F
F2_UsCarrier=res_2_UsCarrier.F
F2_HiberniaGlobal=res_2_HiberniaGlobal.F
F2_Colt=res_2_Colt.F
F2_Funet=res_2_Funet.F
F2_Abvt=res_2_Abvt.F
F2_Intellifiber=res_2_Intellifiber.F
F2_TataNld=res_2_TataNld.F
F2_Internode=res_2_Internode.F
F2_Missouri=res_2_Missouri.F
F2_Ion=res_2_Ion.F
F2_Palmetto=res_2_Palmetto.F

# nsga2
F3_Cogent=res_3_Cogent.F
F3_UsCarrier=res_3_UsCarrier.F
F3_HiberniaGlobal=res_3_HiberniaGlobal.F
F3_Colt=res_3_Colt.F
F3_Funet=res_3_Funet.F
F3_Abvt=res_3_Abvt.F
F3_Intellifiber=res_3_Intellifiber.F
F3_TataNld=res_3_TataNld.F
F3_Internode=res_3_Internode.F
F3_Missouri=res_3_Missouri.F
F3_Ion=res_3_Ion.F
F3_Palmetto=res_3_Palmetto.F

# nsga3
F4_Cogent=res_4_Cogent.F
F4_UsCarrier=res_4_UsCarrier.F
F4_HiberniaGlobal=res_4_HiberniaGlobal.F
F4_Colt=res_4_Colt.F
F4_Funet=res_4_Funet.F
F4_Abvt=res_4_Abvt.F
F4_Intellifiber=res_4_Intellifiber.F
F4_TataNld=res_4_TataNld.F
F4_Internode=res_4_Internode.F
F4_Missouri=res_4_Missouri.F
F4_Ion=res_4_Ion.F
F4_Palmetto=res_4_Palmetto.F

# rnsga2
F5_Cogent=res_5_Cogent.F
F5_UsCarrier=res_5_UsCarrier.F
F5_HiberniaGlobal=res_5_HiberniaGlobal.F
F5_Colt=res_5_Colt.F
F5_Funet=res_5_Funet.F
F5_Abvt=res_5_Abvt.F
F5_Intellifiber=res_5_Intellifiber.F
F5_TataNld=res_5_TataNld.F
F5_Internode=res_5_Internode.F
F5_Missouri=res_5_Missouri.F
F5_Ion=res_5_Ion.F
F5_Palmetto=res_5_Palmetto.F

# rnsga3
F6_Cogent=res_6_Cogent.F
F6_UsCarrier=res_6_UsCarrier.F
F6_HiberniaGlobal=res_6_HiberniaGlobal.F
F6_Colt=res_6_Colt.F
F6_Funet=res_6_Funet.F
F6_Abvt=res_6_Abvt.F
F6_Intellifiber=res_6_Intellifiber.F
F6_TataNld=res_6_TataNld.F
F6_Internode=res_6_Internode.F
F6_Missouri=res_6_Missouri.F
F6_Ion=res_6_Ion.F
F6_Palmetto=res_6_Palmetto.F

# rvea
F7_Cogent=res_7_Cogent.F
F7_UsCarrier=res_7_UsCarrier.F
F7_HiberniaGlobal=res_7_HiberniaGlobal.F
F7_Colt=res_7_Colt.F
F7_Funet=res_7_Funet.F
F7_Abvt=res_7_Abvt.F
F7_Intellifiber=res_7_Intellifiber.F
F7_TataNld=res_7_TataNld.F
F7_Internode=res_7_Internode.F
F7_Missouri=res_7_Missouri.F
F7_Ion=res_7_Ion.F
F7_Palmetto=res_7_Palmetto.F

# smsemoa
F8_Cogent=res_8_Cogent.F
F8_UsCarrier=res_8_UsCarrier.F
F8_HiberniaGlobal=res_8_HiberniaGlobal.F
F8_Colt=res_8_Colt.F
F8_Funet=res_8_Funet.F
F8_Abvt=res_8_Abvt.F
F8_Intellifiber=res_8_Intellifiber.F
F8_TataNld=res_8_TataNld.F
F8_Internode=res_8_Internode.F
F8_Missouri=res_8_Missouri.F
F8_Ion=res_8_Ion.F
F8_Palmetto=res_8_Palmetto.F

# unsga3
F9_Cogent=res_9_Cogent.F
F9_UsCarrier=res_9_UsCarrier.F
F9_HiberniaGlobal=res_9_HiberniaGlobal.F
F9_Colt=res_9_Colt.F
F9_Funet=res_9_Funet.F
F9_Abvt=res_9_Abvt.F
F9_Intellifiber=res_9_Intellifiber.F
F9_TataNld=res_9_TataNld.F
F9_Internode=res_9_Internode.F
F9_Missouri=res_9_Missouri.F
F9_Ion=res_9_Ion.F
F9_Palmetto=res_9_Palmetto.F

# ctaea
F10_Cogent=res_10_Cogent.F
F10_UsCarrier=res_10_UsCarrier.F
F10_HiberniaGlobal=res_10_HiberniaGlobal.F
F10_Colt=res_10_Colt.F
F10_Funet=res_10_Funet.F
F10_Abvt=res_10_Abvt.F
F10_Intellifiber=res_10_Intellifiber.F
F10_TataNld=res_10_TataNld.F
F10_Internode=res_10_Internode.F
F10_Missouri=res_10_Missouri.F
F10_Ion=res_10_Ion.F
F10_Palmetto=res_10_Palmetto.F

# dnsga2
F11_Cogent=res_11_Cogent.F
F11_UsCarrier=res_11_UsCarrier.F
F11_HiberniaGlobal=res_11_HiberniaGlobal.F
F11_Colt=res_11_Colt.F
F11_Funet=res_11_Funet.F
F11_Abvt=res_11_Abvt.F
F11_Intellifiber=res_11_Intellifiber.F
F11_TataNld=res_11_TataNld.F
F11_Internode=res_11_Internode.F
F11_Missouri=res_11_Missouri.F
F11_Ion=res_11_Ion.F
F11_Palmetto=res_11_Palmetto.F

# kgbdmoea
F12_Cogent=res_12_Cogent.F
F12_UsCarrier=res_12_UsCarrier.F
F12_HiberniaGlobal=res_12_HiberniaGlobal.F
F12_Colt=res_12_Colt.F
F12_Funet=res_12_Funet.F
F12_Abvt=res_12_Abvt.F
F12_Intellifiber=res_12_Intellifiber.F
F12_TataNld=res_12_TataNld.F
F12_Internode=res_12_Internode.F
F12_Missouri=res_12_Missouri.F
F12_Ion=res_12_Ion.F
F12_Palmetto=res_12_Palmetto.F

# moead
F13_Cogent=res_13_Cogent.F
F13_UsCarrier=res_13_UsCarrier.F
F13_HiberniaGlobal=res_13_HiberniaGlobal.F
F13_Colt=res_13_Colt.F
F13_Funet=res_13_Funet.F
F13_Abvt=res_13_Abvt.F
F13_Intellifiber=res_13_Intellifiber.F
F13_TataNld=res_13_TataNld.F
F13_Internode=res_13_Internode.F
F13_Missouri=res_13_Missouri.F
F13_Ion=res_13_Ion.F
F13_Palmetto=res_13_Palmetto.F



# # Nadir Point from 9 Algorithms: 
# ref_point = [1.81000000e+02 6.20000000e+01 3.92281875e+03 1.70537209e+00]
# # Nadir Point from 4 Algorithms: 
# ref_point = [160.  77.  inf  inf]
# # Nadir Point from all (13) Algorithms: 
# ref_point = [181.  77.  inf  inf]

ref_point = [181, 77, 3922.81875, 1.70537209]

hist_F1_Cogent = []
hist_F1_UsCarrier = []
hist_F1_HiberniaGlobal = []
hist_F1_Colt = []
hist_F1_Funet = []
hist_F1_Abvt = []
hist_F1_Intellifiber = []
hist_F1_TataNld = []
hist_F1_Internode = []
hist_F1_Missouri = []
hist_F1_Ion = []
hist_F1_Palmetto = []

hist_F2_Cogent = []
hist_F2_UsCarrier = []
hist_F2_HiberniaGlobal = []
hist_F2_Colt = []
hist_F2_Funet = []
hist_F2_Abvt = []
hist_F2_Intellifiber = []
hist_F2_TataNld = []
hist_F2_Internode = []
hist_F2_Missouri = []
hist_F2_Ion = []
hist_F2_Palmetto = []

hist_F3_Cogent = []
hist_F3_UsCarrier = []
hist_F3_HiberniaGlobal = []
hist_F3_Colt = []
hist_F3_Funet = []
hist_F3_Abvt = []
hist_F3_Intellifiber = []
hist_F3_TataNld = []
hist_F3_Internode = []
hist_F3_Missouri = []
hist_F3_Ion = []
hist_F3_Palmetto = []

hist_F4_Cogent = []
hist_F4_UsCarrier = []
hist_F4_HiberniaGlobal = []
hist_F4_Colt = []
hist_F4_Funet = []
hist_F4_Abvt = []
hist_F4_Intellifiber = []
hist_F4_TataNld = []
hist_F4_Internode = []
hist_F4_Missouri = []
hist_F4_Ion = []
hist_F4_Palmetto = []

hist_F5_Cogent = []
hist_F5_UsCarrier = []
hist_F5_HiberniaGlobal = []
hist_F5_Colt = []
hist_F5_Funet = []
hist_F5_Abvt = []
hist_F5_Intellifiber = []
hist_F5_TataNld = []
hist_F5_Internode = []
hist_F5_Missouri = []
hist_F5_Ion = []
hist_F5_Palmetto = []

hist_F6_Cogent = []
hist_F6_UsCarrier = []
hist_F6_HiberniaGlobal = []
hist_F6_Colt = []
hist_F6_Funet = []
hist_F6_Abvt = []
hist_F6_Intellifiber = []
hist_F6_TataNld = []
hist_F6_Internode = []
hist_F6_Missouri = []
hist_F6_Ion = []
hist_F6_Palmetto = []

hist_F7_Cogent = []
hist_F7_UsCarrier = []
hist_F7_HiberniaGlobal = []
hist_F7_Colt = []
hist_F7_Funet = []
hist_F7_Abvt = []
hist_F7_Intellifiber = []
hist_F7_TataNld = []
hist_F7_Internode = []
hist_F7_Missouri = []
hist_F7_Ion = []
hist_F7_Palmetto = []

hist_F8_Cogent = []
hist_F8_UsCarrier = []
hist_F8_HiberniaGlobal = []
hist_F8_Colt = []
hist_F8_Funet = []
hist_F8_Abvt = []
hist_F8_Intellifiber = []
hist_F8_TataNld = []
hist_F8_Internode = []
hist_F8_Missouri = []
hist_F8_Ion = []
hist_F8_Palmetto = []

hist_F9_Cogent = []
hist_F9_UsCarrier = []
hist_F9_HiberniaGlobal = []
hist_F9_Colt = []
hist_F9_Funet = []
hist_F9_Abvt = []
hist_F9_Intellifiber = []
hist_F9_TataNld = []
hist_F9_Internode = []
hist_F9_Missouri = []
hist_F9_Ion = []
hist_F9_Palmetto = []

hist_F10_Cogent = []
hist_F10_UsCarrier = []
hist_F10_HiberniaGlobal = []
hist_F10_Colt = []
hist_F10_Funet = []
hist_F10_Abvt = []
hist_F10_Intellifiber = []
hist_F10_TataNld = []
hist_F10_Internode = []
hist_F10_Missouri = []
hist_F10_Ion = []
hist_F10_Palmetto = []

hist_F11_Cogent = []
hist_F11_UsCarrier = []
hist_F11_HiberniaGlobal = []
hist_F11_Colt = []
hist_F11_Funet = []
hist_F11_Abvt = []
hist_F11_Intellifiber = []
hist_F11_TataNld = []
hist_F11_Internode = []
hist_F11_Missouri = []
hist_F11_Ion = []
hist_F11_Palmetto = []

hist_F12_Cogent = []
hist_F12_UsCarrier = []
hist_F12_HiberniaGlobal = []
hist_F12_Colt = []
hist_F12_Funet = []
hist_F12_Abvt = []
hist_F12_Intellifiber = []
hist_F12_TataNld = []
hist_F12_Internode = []
hist_F12_Missouri = []
hist_F12_Ion = []
hist_F12_Palmetto = []

hist_F13_Cogent = []
hist_F13_UsCarrier = []
hist_F13_HiberniaGlobal = []
hist_F13_Colt = []
hist_F13_Funet = []
hist_F13_Abvt = []
hist_F13_Intellifiber = []
hist_F13_TataNld = []
hist_F13_Internode = []
hist_F13_Missouri = []
hist_F13_Ion = []
hist_F13_Palmetto = []

for algo1_Cogent in res_1_Cogent.history:
    opt1_Cogent = algo1_Cogent.opt
    feas1_Cogent = np.where(opt1_Cogent.get("feasible"))[0]
    hist_F1_Cogent.append(opt1_Cogent.get("F")[feas1_Cogent])
for algo1_UsCarrier in res_1_UsCarrier.history:
    opt1_UsCarrier = algo1_UsCarrier.opt
    feas1_UsCarrier = np.where(opt1_UsCarrier.get("feasible"))[0]
    hist_F1_UsCarrier.append(opt1_UsCarrier.get("F")[feas1_UsCarrier])
for algo1_HiberniaGlobal in res_1_HiberniaGlobal.history:
    opt1_HiberniaGlobal = algo1_HiberniaGlobal.opt
    feas1_HiberniaGlobal = np.where(opt1_HiberniaGlobal.get("feasible"))[0]
    hist_F1_HiberniaGlobal.append(opt1_HiberniaGlobal.get("F")[feas1_HiberniaGlobal])
for algo1_Colt in res_1_Colt.history:
    opt1_Colt = algo1_Colt.opt
    feas1_Colt = np.where(opt1_Colt.get("feasible"))[0]
    hist_F1_Colt.append(opt1_Colt.get("F")[feas1_Colt])
for algo1_Funet in res_1_Funet.history:
    opt1_Funet = algo1_Funet.opt
    feas1_Funet = np.where(opt1_Funet.get("feasible"))[0]
    hist_F1_Funet.append(opt1_Funet.get("F")[feas1_Funet])
for algo1_Abvt in res_1_Abvt.history:
    opt1_Abvt = algo1_Abvt.opt
    feas1_Abvt = np.where(opt1_Abvt.get("feasible"))[0]
    hist_F1_Abvt.append(opt1_Abvt.get("F")[feas1_Abvt])
for algo1_Intellifiber in res_1_Intellifiber.history:
    opt1_Intellifiber = algo1_Intellifiber.opt
    feas1_Intellifiber = np.where(opt1_Intellifiber.get("feasible"))[0]
    hist_F1_Intellifiber.append(opt1_Intellifiber.get("F")[feas1_Intellifiber])
for algo1_TataNld in res_1_TataNld.history:
    opt1_TataNld = algo1_TataNld.opt
    feas1_TataNld = np.where(opt1_TataNld.get("feasible"))[0]
    hist_F1_TataNld.append(opt1_TataNld.get("F")[feas1_TataNld])
for algo1_Internode in res_1_Internode.history:
    opt1_Internode = algo1_Internode.opt
    feas1_Internode= np.where(opt1_Internode.get("feasible"))[0]
    hist_F1_Internode.append(opt1_Internode.get("F")[feas1_Internode])
for algo1_Missouri in res_1_Missouri.history:
    opt1_Missouri = algo1_Missouri.opt
    feas1_Missouri = np.where(opt1_Missouri.get("feasible"))[0]
    hist_F1_Missouri.append(opt1_Missouri.get("F")[feas1_Missouri])
for algo1_Ion in res_1_Ion.history:
    opt1_Ion = algo1_Ion.opt
    feas1_Ion = np.where(opt1_Ion.get("feasible"))[0]
    hist_F1_Ion.append(opt1_Ion.get("F")[feas1_Ion])
for algo1_Palmetto in res_1_Palmetto.history:
    opt1_Palmetto = algo1_Palmetto.opt
    feas1_Palmetto = np.where(opt1_Palmetto.get("feasible"))[0]
    hist_F1_Palmetto.append(opt1_Palmetto.get("F")[feas1_Palmetto])



for algo2_Cogent in res_2_Cogent.history:
    opt2_Cogent = algo2_Cogent.opt
    feas2_Cogent = np.where(opt2_Cogent.get("feasible"))[0]
    hist_F2_Cogent.append(opt2_Cogent.get("F")[feas2_Cogent])
for algo2_UsCarrier in res_2_UsCarrier.history:
    opt2_UsCarrier = algo2_UsCarrier.opt
    feas2_UsCarrier = np.where(opt2_UsCarrier.get("feasible"))[0]
    hist_F2_UsCarrier.append(opt2_UsCarrier.get("F")[feas2_UsCarrier])
for algo2_HiberniaGlobal in res_2_HiberniaGlobal.history:
    opt2_HiberniaGlobal = algo2_HiberniaGlobal.opt
    feas2_HiberniaGlobal = np.where(opt2_HiberniaGlobal.get("feasible"))[0]
    hist_F2_HiberniaGlobal.append(opt2_HiberniaGlobal.get("F")[feas2_HiberniaGlobal])
for algo2_Colt in res_2_Colt.history:
    opt2_Colt = algo2_Colt.opt
    feas2_Colt = np.where(opt2_Colt.get("feasible"))[0]
    hist_F2_Colt.append(opt2_Colt.get("F")[feas2_Colt])
for algo2_Funet in res_2_Funet.history:
    opt2_Funet = algo2_Funet.opt
    feas2_Funet = np.where(opt2_Funet.get("feasible"))[0]
    hist_F2_Funet.append(opt2_Funet.get("F")[feas2_Funet])
for algo2_Abvt in res_2_Abvt.history:
    opt2_Abvt = algo2_Abvt.opt
    feas2_Abvt = np.where(opt2_Abvt.get("feasible"))[0]
    hist_F2_Abvt.append(opt2_Abvt.get("F")[feas2_Abvt])
for algo2_Intellifiber in res_2_Intellifiber.history:
    opt2_Intellifiber = algo2_Intellifiber.opt
    feas2_Intellifiber = np.where(opt2_Intellifiber.get("feasible"))[0]
    hist_F2_Intellifiber.append(opt2_Intellifiber.get("F")[feas2_Intellifiber])
for algo2_TataNld in res_2_TataNld.history:
    opt2_TataNld = algo2_TataNld.opt
    feas2_TataNld = np.where(opt2_TataNld.get("feasible"))[0]
    hist_F2_TataNld.append(opt2_TataNld.get("F")[feas2_TataNld])
for algo2_Internode in res_2_Internode.history:
    opt2_Internode = algo2_Internode.opt
    feas2_Internode= np.where(opt2_Internode.get("feasible"))[0]
    hist_F2_Internode.append(opt2_Internode.get("F")[feas2_Internode])
for algo2_Missouri in res_2_Missouri.history:
    opt2_Missouri = algo2_Missouri.opt
    feas2_Missouri = np.where(opt2_Missouri.get("feasible"))[0]
    hist_F2_Missouri.append(opt2_Missouri.get("F")[feas2_Missouri])
for algo2_Ion in res_2_Ion.history:
    opt2_Ion = algo2_Ion.opt
    feas2_Ion = np.where(opt2_Ion.get("feasible"))[0]
    hist_F2_Ion.append(opt2_Ion.get("F")[feas2_Ion])
for algo2_Palmetto in res_2_Palmetto.history:
    opt2_Palmetto = algo2_Palmetto.opt
    feas2_Palmetto = np.where(opt2_Palmetto.get("feasible"))[0]
    hist_F2_Palmetto.append(opt2_Palmetto.get("F")[feas2_Palmetto])

for algo3_Cogent in res_3_Cogent.history:
    opt3_Cogent = algo3_Cogent.opt
    feas3_Cogent = np.where(opt3_Cogent.get("feasible"))[0]
    hist_F3_Cogent.append(opt3_Cogent.get("F")[feas3_Cogent])
for algo3_UsCarrier in res_3_UsCarrier.history:
    opt3_UsCarrier = algo3_UsCarrier.opt
    feas3_UsCarrier = np.where(opt3_UsCarrier.get("feasible"))[0]
    hist_F3_UsCarrier.append(opt3_UsCarrier.get("F")[feas3_UsCarrier])
for algo3_HiberniaGlobal in res_3_HiberniaGlobal.history:
    opt3_HiberniaGlobal = algo3_HiberniaGlobal.opt
    feas3_HiberniaGlobal = np.where(opt3_HiberniaGlobal.get("feasible"))[0]
    hist_F3_HiberniaGlobal.append(opt3_HiberniaGlobal.get("F")[feas3_HiberniaGlobal])
for algo3_Colt in res_3_Colt.history:
    opt3_Colt = algo3_Colt.opt
    feas3_Colt = np.where(opt3_Colt.get("feasible"))[0]
    hist_F3_Colt.append(opt3_Colt.get("F")[feas3_Colt])
for algo3_Funet in res_3_Funet.history:
    opt3_Funet = algo3_Funet.opt
    feas3_Funet = np.where(opt3_Funet.get("feasible"))[0]
    hist_F3_Funet.append(opt3_Funet.get("F")[feas3_Funet])
for algo3_Abvt in res_3_Abvt.history:
    opt3_Abvt = algo3_Abvt.opt
    feas3_Abvt = np.where(opt3_Abvt.get("feasible"))[0]
    hist_F3_Abvt.append(opt3_Abvt.get("F")[feas3_Abvt])
for algo3_Intellifiber in res_3_Intellifiber.history:
    opt3_Intellifiber = algo3_Intellifiber.opt
    feas3_Intellifiber = np.where(opt3_Intellifiber.get("feasible"))[0]
    hist_F3_Intellifiber.append(opt3_Intellifiber.get("F")[feas3_Intellifiber])
for algo3_TataNld in res_3_TataNld.history:
    opt3_TataNld = algo3_TataNld.opt
    feas3_TataNld = np.where(opt3_TataNld.get("feasible"))[0]
    hist_F3_TataNld.append(opt3_TataNld.get("F")[feas3_TataNld])
for algo3_Internode in res_3_Internode.history:
    opt3_Internode = algo3_Internode.opt
    feas3_Internode= np.where(opt3_Internode.get("feasible"))[0]
    hist_F3_Internode.append(opt3_Internode.get("F")[feas3_Internode])
for algo3_Missouri in res_3_Missouri.history:
    opt3_Missouri = algo3_Missouri.opt
    feas3_Missouri = np.where(opt3_Missouri.get("feasible"))[0]
    hist_F3_Missouri.append(opt3_Missouri.get("F")[feas3_Missouri])
for algo3_Ion in res_3_Ion.history:
    opt3_Ion = algo3_Ion.opt
    feas3_Ion = np.where(opt3_Ion.get("feasible"))[0]
    hist_F3_Ion.append(opt3_Ion.get("F")[feas3_Ion])
for algo3_Palmetto in res_3_Palmetto.history:
    opt3_Palmetto = algo3_Palmetto.opt
    feas3_Palmetto = np.where(opt3_Palmetto.get("feasible"))[0]
    hist_F3_Palmetto.append(opt3_Palmetto.get("F")[feas3_Palmetto])

for algo4_Cogent in res_4_Cogent.history:
    opt4_Cogent = algo4_Cogent.opt
    feas4_Cogent = np.where(opt4_Cogent.get("feasible"))[0]
    hist_F4_Cogent.append(opt4_Cogent.get("F")[feas4_Cogent])
for algo4_UsCarrier in res_4_UsCarrier.history:
    opt4_UsCarrier = algo4_UsCarrier.opt
    feas4_UsCarrier = np.where(opt4_UsCarrier.get("feasible"))[0]
    hist_F4_UsCarrier.append(opt4_UsCarrier.get("F")[feas4_UsCarrier])
for algo4_HiberniaGlobal in res_4_HiberniaGlobal.history:
    opt4_HiberniaGlobal = algo4_HiberniaGlobal.opt
    feas4_HiberniaGlobal = np.where(opt4_HiberniaGlobal.get("feasible"))[0]
    hist_F4_HiberniaGlobal.append(opt4_HiberniaGlobal.get("F")[feas4_HiberniaGlobal])
for algo4_Colt in res_4_Colt.history:
    opt4_Colt = algo4_Colt.opt
    feas4_Colt = np.where(opt4_Colt.get("feasible"))[0]
    hist_F4_Colt.append(opt4_Colt.get("F")[feas4_Colt])
for algo4_Funet in res_4_Funet.history:
    opt4_Funet = algo4_Funet.opt
    feas4_Funet = np.where(opt4_Funet.get("feasible"))[0]
    hist_F4_Funet.append(opt4_Funet.get("F")[feas4_Funet])
for algo4_Abvt in res_4_Abvt.history:
    opt4_Abvt = algo4_Abvt.opt
    feas4_Abvt = np.where(opt4_Abvt.get("feasible"))[0]
    hist_F4_Abvt.append(opt4_Abvt.get("F")[feas4_Abvt])
for algo4_Intellifiber in res_4_Intellifiber.history:
    opt4_Intellifiber = algo4_Intellifiber.opt
    feas4_Intellifiber = np.where(opt4_Intellifiber.get("feasible"))[0]
    hist_F4_Intellifiber.append(opt4_Intellifiber.get("F")[feas4_Intellifiber])
for algo4_TataNld in res_4_TataNld.history:
    opt4_TataNld = algo4_TataNld.opt
    feas4_TataNld = np.where(opt4_TataNld.get("feasible"))[0]
    hist_F4_TataNld.append(opt4_TataNld.get("F")[feas4_TataNld])
for algo4_Internode in res_4_Internode.history:
    opt4_Internode = algo4_Internode.opt
    feas4_Internode= np.where(opt4_Internode.get("feasible"))[0]
    hist_F4_Internode.append(opt4_Internode.get("F")[feas4_Internode])
for algo4_Missouri in res_4_Missouri.history:
    opt4_Missouri = algo4_Missouri.opt
    feas4_Missouri = np.where(opt4_Missouri.get("feasible"))[0]
    hist_F4_Missouri.append(opt4_Missouri.get("F")[feas4_Missouri])
for algo4_Ion in res_4_Ion.history:
    opt4_Ion = algo4_Ion.opt
    feas4_Ion = np.where(opt4_Ion.get("feasible"))[0]
    hist_F4_Ion.append(opt4_Ion.get("F")[feas4_Ion])
for algo4_Palmetto in res_4_Palmetto.history:
    opt4_Palmetto = algo4_Palmetto.opt
    feas4_Palmetto = np.where(opt4_Palmetto.get("feasible"))[0]
    hist_F4_Palmetto.append(opt4_Palmetto.get("F")[feas4_Palmetto])

for algo5_Cogent in res_5_Cogent.history:
    opt5_Cogent = algo5_Cogent.opt
    feas5_Cogent = np.where(opt5_Cogent.get("feasible"))[0]
    hist_F5_Cogent.append(opt5_Cogent.get("F")[feas5_Cogent])
for algo5_UsCarrier in res_5_UsCarrier.history:
    opt5_UsCarrier = algo5_UsCarrier.opt
    feas5_UsCarrier = np.where(opt5_UsCarrier.get("feasible"))[0]
    hist_F5_UsCarrier.append(opt5_UsCarrier.get("F")[feas5_UsCarrier])
for algo5_HiberniaGlobal in res_5_HiberniaGlobal.history:
    opt5_HiberniaGlobal = algo5_HiberniaGlobal.opt
    feas5_HiberniaGlobal = np.where(opt5_HiberniaGlobal.get("feasible"))[0]
    hist_F5_HiberniaGlobal.append(opt5_HiberniaGlobal.get("F")[feas5_HiberniaGlobal])
for algo5_Colt in res_5_Colt.history:
    opt5_Colt = algo5_Colt.opt
    feas5_Colt = np.where(opt5_Colt.get("feasible"))[0]
    hist_F5_Colt.append(opt5_Colt.get("F")[feas5_Colt])
for algo5_Funet in res_5_Funet.history:
    opt5_Funet = algo5_Funet.opt
    feas5_Funet = np.where(opt5_Funet.get("feasible"))[0]
    hist_F5_Funet.append(opt5_Funet.get("F")[feas5_Funet])
for algo5_Abvt in res_5_Abvt.history:
    opt5_Abvt = algo5_Abvt.opt
    feas5_Abvt = np.where(opt5_Abvt.get("feasible"))[0]
    hist_F5_Abvt.append(opt5_Abvt.get("F")[feas5_Abvt])
for algo5_Intellifiber in res_5_Intellifiber.history:
    opt5_Intellifiber = algo5_Intellifiber.opt
    feas5_Intellifiber = np.where(opt5_Intellifiber.get("feasible"))[0]
    hist_F5_Intellifiber.append(opt5_Intellifiber.get("F")[feas5_Intellifiber])
for algo5_TataNld in res_5_TataNld.history:
    opt5_TataNld = algo5_TataNld.opt
    feas5_TataNld = np.where(opt5_TataNld.get("feasible"))[0]
    hist_F5_TataNld.append(opt5_TataNld.get("F")[feas5_TataNld])
for algo5_Internode in res_5_Internode.history:
    opt5_Internode = algo5_Internode.opt
    feas5_Internode= np.where(opt5_Internode.get("feasible"))[0]
    hist_F5_Internode.append(opt5_Internode.get("F")[feas5_Internode])
for algo5_Missouri in res_5_Missouri.history:
    opt5_Missouri = algo5_Missouri.opt
    feas5_Missouri = np.where(opt5_Missouri.get("feasible"))[0]
    hist_F5_Missouri.append(opt5_Missouri.get("F")[feas5_Missouri])
for algo5_Ion in res_5_Ion.history:
    opt5_Ion = algo5_Ion.opt
    feas5_Ion = np.where(opt5_Ion.get("feasible"))[0]
    hist_F5_Ion.append(opt5_Ion.get("F")[feas5_Ion])
for algo5_Palmetto in res_5_Palmetto.history:
    opt5_Palmetto = algo5_Palmetto.opt
    feas5_Palmetto = np.where(opt5_Palmetto.get("feasible"))[0]
    hist_F5_Palmetto.append(opt5_Palmetto.get("F")[feas5_Palmetto])

for algo6_Cogent in res_6_Cogent.history:
    opt6_Cogent = algo6_Cogent.opt
    feas6_Cogent = np.where(opt6_Cogent.get("feasible"))[0]
    hist_F6_Cogent.append(opt6_Cogent.get("F")[feas6_Cogent])
for algo6_UsCarrier in res_6_UsCarrier.history:
    opt6_UsCarrier = algo6_UsCarrier.opt
    feas6_UsCarrier = np.where(opt6_UsCarrier.get("feasible"))[0]
    hist_F6_UsCarrier.append(opt6_UsCarrier.get("F")[feas6_UsCarrier])
for algo6_HiberniaGlobal in res_6_HiberniaGlobal.history:
    opt6_HiberniaGlobal = algo6_HiberniaGlobal.opt
    feas6_HiberniaGlobal = np.where(opt6_HiberniaGlobal.get("feasible"))[0]
    hist_F6_HiberniaGlobal.append(opt6_HiberniaGlobal.get("F")[feas6_HiberniaGlobal])
for algo6_Colt in res_6_Colt.history:
    opt6_Colt = algo6_Colt.opt
    feas6_Colt = np.where(opt6_Colt.get("feasible"))[0]
    hist_F6_Colt.append(opt6_Colt.get("F")[feas6_Colt])
for algo6_Funet in res_6_Funet.history:
    opt6_Funet = algo6_Funet.opt
    feas6_Funet = np.where(opt6_Funet.get("feasible"))[0]
    hist_F6_Funet.append(opt6_Funet.get("F")[feas6_Funet])
for algo6_Abvt in res_6_Abvt.history:
    opt6_Abvt = algo6_Abvt.opt
    feas6_Abvt = np.where(opt6_Abvt.get("feasible"))[0]
    hist_F6_Abvt.append(opt6_Abvt.get("F")[feas6_Abvt])
for algo6_Intellifiber in res_6_Intellifiber.history:
    opt6_Intellifiber = algo6_Intellifiber.opt
    feas6_Intellifiber = np.where(opt6_Intellifiber.get("feasible"))[0]
    hist_F6_Intellifiber.append(opt6_Intellifiber.get("F")[feas6_Intellifiber])
for algo6_TataNld in res_6_TataNld.history:
    opt6_TataNld = algo6_TataNld.opt
    feas6_TataNld = np.where(opt6_TataNld.get("feasible"))[0]
    hist_F6_TataNld.append(opt6_TataNld.get("F")[feas6_TataNld])
for algo6_Internode in res_6_Internode.history:
    opt6_Internode = algo6_Internode.opt
    feas6_Internode= np.where(opt6_Internode.get("feasible"))[0]
    hist_F6_Internode.append(opt6_Internode.get("F")[feas6_Internode])
for algo6_Missouri in res_6_Missouri.history:
    opt6_Missouri = algo6_Missouri.opt
    feas6_Missouri = np.where(opt6_Missouri.get("feasible"))[0]
    hist_F6_Missouri.append(opt6_Missouri.get("F")[feas6_Missouri])
for algo6_Ion in res_6_Ion.history:
    opt6_Ion = algo6_Ion.opt
    feas6_Ion = np.where(opt6_Ion.get("feasible"))[0]
    hist_F6_Ion.append(opt6_Ion.get("F")[feas6_Ion])
for algo6_Palmetto in res_6_Palmetto.history:
    opt6_Palmetto = algo6_Palmetto.opt
    feas6_Palmetto = np.where(opt6_Palmetto.get("feasible"))[0]
    hist_F6_Palmetto.append(opt6_Palmetto.get("F")[feas6_Palmetto])

for algo7_Cogent in res_7_Cogent.history:
    opt7_Cogent = algo7_Cogent.opt
    feas7_Cogent = np.where(opt7_Cogent.get("feasible"))[0]
    hist_F7_Cogent.append(opt7_Cogent.get("F")[feas7_Cogent])
for algo7_UsCarrier in res_7_UsCarrier.history:
    opt7_UsCarrier = algo7_UsCarrier.opt
    feas7_UsCarrier = np.where(opt7_UsCarrier.get("feasible"))[0]
    hist_F7_UsCarrier.append(opt7_UsCarrier.get("F")[feas7_UsCarrier])
for algo7_HiberniaGlobal in res_7_HiberniaGlobal.history:
    opt7_HiberniaGlobal = algo7_HiberniaGlobal.opt
    feas7_HiberniaGlobal = np.where(opt7_HiberniaGlobal.get("feasible"))[0]
    hist_F7_HiberniaGlobal.append(opt7_HiberniaGlobal.get("F")[feas7_HiberniaGlobal])
for algo7_Colt in res_7_Colt.history:
    opt7_Colt = algo7_Colt.opt
    feas7_Colt = np.where(opt7_Colt.get("feasible"))[0]
    hist_F7_Colt.append(opt7_Colt.get("F")[feas7_Colt])
for algo7_Funet in res_7_Funet.history:
    opt7_Funet = algo7_Funet.opt
    feas7_Funet = np.where(opt7_Funet.get("feasible"))[0]
    hist_F7_Funet.append(opt7_Funet.get("F")[feas7_Funet])
for algo7_Abvt in res_7_Abvt.history:
    opt7_Abvt = algo7_Abvt.opt
    feas7_Abvt = np.where(opt7_Abvt.get("feasible"))[0]
    hist_F7_Abvt.append(opt7_Abvt.get("F")[feas7_Abvt])
for algo7_Intellifiber in res_7_Intellifiber.history:
    opt7_Intellifiber = algo7_Intellifiber.opt
    feas7_Intellifiber = np.where(opt7_Intellifiber.get("feasible"))[0]
    hist_F7_Intellifiber.append(opt7_Intellifiber.get("F")[feas7_Intellifiber])
for algo7_TataNld in res_7_TataNld.history:
    opt7_TataNld = algo7_TataNld.opt
    feas7_TataNld = np.where(opt7_TataNld.get("feasible"))[0]
    hist_F7_TataNld.append(opt7_TataNld.get("F")[feas7_TataNld])
for algo7_Internode in res_7_Internode.history:
    opt7_Internode = algo7_Internode.opt
    feas7_Internode= np.where(opt7_Internode.get("feasible"))[0]
    hist_F7_Internode.append(opt7_Internode.get("F")[feas7_Internode])
for algo7_Missouri in res_7_Missouri.history:
    opt7_Missouri = algo7_Missouri.opt
    feas7_Missouri = np.where(opt7_Missouri.get("feasible"))[0]
    hist_F7_Missouri.append(opt7_Missouri.get("F")[feas7_Missouri])
for algo7_Ion in res_7_Ion.history:
    opt7_Ion = algo7_Ion.opt
    feas7_Ion = np.where(opt7_Ion.get("feasible"))[0]
    hist_F7_Ion.append(opt7_Ion.get("F")[feas7_Ion])
for algo7_Palmetto in res_7_Palmetto.history:
    opt7_Palmetto = algo7_Palmetto.opt
    feas7_Palmetto = np.where(opt7_Palmetto.get("feasible"))[0]
    hist_F7_Palmetto.append(opt7_Palmetto.get("F")[feas7_Palmetto])

for algo8_Cogent in res_8_Cogent.history:
    opt8_Cogent = algo8_Cogent.opt
    feas8_Cogent = np.where(opt8_Cogent.get("feasible"))[0]
    hist_F8_Cogent.append(opt8_Cogent.get("F")[feas8_Cogent])
for algo8_UsCarrier in res_8_UsCarrier.history:
    opt8_UsCarrier = algo8_UsCarrier.opt
    feas8_UsCarrier = np.where(opt8_UsCarrier.get("feasible"))[0]
    hist_F8_UsCarrier.append(opt8_UsCarrier.get("F")[feas8_UsCarrier])
for algo8_HiberniaGlobal in res_8_HiberniaGlobal.history:
    opt8_HiberniaGlobal = algo8_HiberniaGlobal.opt
    feas8_HiberniaGlobal = np.where(opt8_HiberniaGlobal.get("feasible"))[0]
    hist_F8_HiberniaGlobal.append(opt8_HiberniaGlobal.get("F")[feas8_HiberniaGlobal])
for algo8_Colt in res_8_Colt.history:
    opt8_Colt = algo8_Colt.opt
    feas8_Colt = np.where(opt8_Colt.get("feasible"))[0]
    hist_F8_Colt.append(opt8_Colt.get("F")[feas8_Colt])
for algo8_Funet in res_8_Funet.history:
    opt8_Funet = algo8_Funet.opt
    feas8_Funet = np.where(opt8_Funet.get("feasible"))[0]
    hist_F8_Funet.append(opt8_Funet.get("F")[feas8_Funet])
for algo8_Abvt in res_8_Abvt.history:
    opt8_Abvt = algo8_Abvt.opt
    feas8_Abvt = np.where(opt8_Abvt.get("feasible"))[0]
    hist_F8_Abvt.append(opt8_Abvt.get("F")[feas8_Abvt])
for algo8_Intellifiber in res_8_Intellifiber.history:
    opt8_Intellifiber = algo8_Intellifiber.opt
    feas8_Intellifiber = np.where(opt8_Intellifiber.get("feasible"))[0]
    hist_F8_Intellifiber.append(opt8_Intellifiber.get("F")[feas8_Intellifiber])
for algo8_TataNld in res_8_TataNld.history:
    opt8_TataNld = algo8_TataNld.opt
    feas8_TataNld = np.where(opt8_TataNld.get("feasible"))[0]
    hist_F8_TataNld.append(opt8_TataNld.get("F")[feas8_TataNld])
for algo8_Internode in res_8_Internode.history:
    opt8_Internode = algo8_Internode.opt
    feas8_Internode= np.where(opt8_Internode.get("feasible"))[0]
    hist_F8_Internode.append(opt8_Internode.get("F")[feas8_Internode])
for algo8_Missouri in res_8_Missouri.history:
    opt8_Missouri = algo8_Missouri.opt
    feas8_Missouri = np.where(opt8_Missouri.get("feasible"))[0]
    hist_F8_Missouri.append(opt8_Missouri.get("F")[feas8_Missouri])
for algo8_Ion in res_8_Ion.history:
    opt8_Ion = algo8_Ion.opt
    feas8_Ion = np.where(opt8_Ion.get("feasible"))[0]
    hist_F8_Ion.append(opt8_Ion.get("F")[feas8_Ion])
for algo8_Palmetto in res_8_Palmetto.history:
    opt8_Palmetto = algo8_Palmetto.opt
    feas8_Palmetto = np.where(opt8_Palmetto.get("feasible"))[0]
    hist_F8_Palmetto.append(opt8_Palmetto.get("F")[feas8_Palmetto])

for algo9_Cogent in res_9_Cogent.history:
    opt9_Cogent = algo9_Cogent.opt
    feas9_Cogent = np.where(opt9_Cogent.get("feasible"))[0]
    hist_F9_Cogent.append(opt9_Cogent.get("F")[feas9_Cogent])
for algo9_UsCarrier in res_9_UsCarrier.history:
    opt9_UsCarrier = algo9_UsCarrier.opt
    feas9_UsCarrier = np.where(opt9_UsCarrier.get("feasible"))[0]
    hist_F9_UsCarrier.append(opt9_UsCarrier.get("F")[feas9_UsCarrier])
for algo9_HiberniaGlobal in res_9_HiberniaGlobal.history:
    opt9_HiberniaGlobal = algo9_HiberniaGlobal.opt
    feas9_HiberniaGlobal = np.where(opt9_HiberniaGlobal.get("feasible"))[0]
    hist_F9_HiberniaGlobal.append(opt9_HiberniaGlobal.get("F")[feas9_HiberniaGlobal])
for algo9_Colt in res_9_Colt.history:
    opt9_Colt = algo9_Colt.opt
    feas9_Colt = np.where(opt9_Colt.get("feasible"))[0]
    hist_F9_Colt.append(opt9_Colt.get("F")[feas9_Colt])
for algo9_Funet in res_9_Funet.history:
    opt9_Funet = algo9_Funet.opt
    feas9_Funet = np.where(opt9_Funet.get("feasible"))[0]
    hist_F9_Funet.append(opt9_Funet.get("F")[feas9_Funet])
for algo9_Abvt in res_9_Abvt.history:
    opt9_Abvt = algo9_Abvt.opt
    feas9_Abvt = np.where(opt9_Abvt.get("feasible"))[0]
    hist_F9_Abvt.append(opt9_Abvt.get("F")[feas9_Abvt])
for algo9_Intellifiber in res_9_Intellifiber.history:
    opt9_Intellifiber = algo9_Intellifiber.opt
    feas9_Intellifiber = np.where(opt9_Intellifiber.get("feasible"))[0]
    hist_F9_Intellifiber.append(opt9_Intellifiber.get("F")[feas9_Intellifiber])
for algo9_TataNld in res_9_TataNld.history:
    opt9_TataNld = algo9_TataNld.opt
    feas9_TataNld = np.where(opt9_TataNld.get("feasible"))[0]
    hist_F9_TataNld.append(opt9_TataNld.get("F")[feas9_TataNld])
for algo9_Internode in res_9_Internode.history:
    opt9_Internode = algo9_Internode.opt
    feas9_Internode= np.where(opt9_Internode.get("feasible"))[0]
    hist_F9_Internode.append(opt9_Internode.get("F")[feas9_Internode])
for algo9_Missouri in res_9_Missouri.history:
    opt9_Missouri = algo9_Missouri.opt
    feas9_Missouri = np.where(opt9_Missouri.get("feasible"))[0]
    hist_F9_Missouri.append(opt9_Missouri.get("F")[feas9_Missouri])
for algo9_Ion in res_9_Ion.history:
    opt9_Ion = algo9_Ion.opt
    feas9_Ion = np.where(opt9_Ion.get("feasible"))[0]
    hist_F9_Ion.append(opt9_Ion.get("F")[feas9_Ion])
for algo9_Palmetto in res_9_Palmetto.history:
    opt9_Palmetto = algo9_Palmetto.opt
    feas9_Palmetto = np.where(opt9_Palmetto.get("feasible"))[0]
    hist_F9_Palmetto.append(opt9_Palmetto.get("F")[feas9_Palmetto])

for algo10_Cogent in res_10_Cogent.history:
    opt10_Cogent = algo10_Cogent.opt
    feas10_Cogent = np.where(opt10_Cogent.get("feasible"))[0]
    hist_F10_Cogent.append(opt10_Cogent.get("F")[feas10_Cogent])
for algo10_UsCarrier in res_10_UsCarrier.history:
    opt10_UsCarrier = algo10_UsCarrier.opt
    feas10_UsCarrier = np.where(opt10_UsCarrier.get("feasible"))[0]
    hist_F10_UsCarrier.append(opt10_UsCarrier.get("F")[feas10_UsCarrier])
for algo10_HiberniaGlobal in res_10_HiberniaGlobal.history:
    opt10_HiberniaGlobal = algo10_HiberniaGlobal.opt
    feas10_HiberniaGlobal = np.where(opt10_HiberniaGlobal.get("feasible"))[0]
    hist_F10_HiberniaGlobal.append(opt10_HiberniaGlobal.get("F")[feas10_HiberniaGlobal])
for algo10_Colt in res_10_Colt.history:
    opt10_Colt = algo10_Colt.opt
    feas10_Colt = np.where(opt10_Colt.get("feasible"))[0]
    hist_F10_Colt.append(opt10_Colt.get("F")[feas10_Colt])
for algo10_Funet in res_10_Funet.history:
    opt10_Funet = algo10_Funet.opt
    feas10_Funet = np.where(opt10_Funet.get("feasible"))[0]
    hist_F10_Funet.append(opt10_Funet.get("F")[feas10_Funet])
for algo10_Abvt in res_10_Abvt.history:
    opt10_Abvt = algo10_Abvt.opt
    feas10_Abvt = np.where(opt10_Abvt.get("feasible"))[0]
    hist_F10_Abvt.append(opt10_Abvt.get("F")[feas10_Abvt])
for algo10_Intellifiber in res_10_Intellifiber.history:
    opt10_Intellifiber = algo10_Intellifiber.opt
    feas10_Intellifiber = np.where(opt10_Intellifiber.get("feasible"))[0]
    hist_F10_Intellifiber.append(opt10_Intellifiber.get("F")[feas10_Intellifiber])
for algo10_TataNld in res_10_TataNld.history:
    opt10_TataNld = algo10_TataNld.opt
    feas10_TataNld = np.where(opt10_TataNld.get("feasible"))[0]
    hist_F10_TataNld.append(opt10_TataNld.get("F")[feas10_TataNld])
for algo10_Internode in res_10_Internode.history:
    opt10_Internode = algo10_Internode.opt
    feas10_Internode= np.where(opt10_Internode.get("feasible"))[0]
    hist_F10_Internode.append(opt10_Internode.get("F")[feas10_Internode])
for algo10_Missouri in res_10_Missouri.history:
    opt10_Missouri = algo10_Missouri.opt
    feas10_Missouri = np.where(opt10_Missouri.get("feasible"))[0]
    hist_F10_Missouri.append(opt10_Missouri.get("F")[feas10_Missouri])
for algo10_Ion in res_10_Ion.history:
    opt10_Ion = algo10_Ion.opt
    feas10_Ion = np.where(opt10_Ion.get("feasible"))[0]
    hist_F10_Ion.append(opt10_Ion.get("F")[feas10_Ion])
for algo10_Palmetto in res_10_Palmetto.history:
    opt10_Palmetto = algo10_Palmetto.opt
    feas10_Palmetto = np.where(opt10_Palmetto.get("feasible"))[0]
    hist_F10_Palmetto.append(opt10_Palmetto.get("F")[feas10_Palmetto])

for algo11_Cogent in res_11_Cogent.history:
    opt11_Cogent = algo11_Cogent.opt
    feas11_Cogent = np.where(opt11_Cogent.get("feasible"))[0]
    hist_F11_Cogent.append(opt11_Cogent.get("F")[feas11_Cogent])
for algo11_UsCarrier in res_11_UsCarrier.history:
    opt11_UsCarrier = algo11_UsCarrier.opt
    feas11_UsCarrier = np.where(opt11_UsCarrier.get("feasible"))[0]
    hist_F11_UsCarrier.append(opt11_UsCarrier.get("F")[feas11_UsCarrier])
for algo11_HiberniaGlobal in res_11_HiberniaGlobal.history:
    opt11_HiberniaGlobal = algo11_HiberniaGlobal.opt
    feas11_HiberniaGlobal = np.where(opt11_HiberniaGlobal.get("feasible"))[0]
    hist_F11_HiberniaGlobal.append(opt11_HiberniaGlobal.get("F")[feas11_HiberniaGlobal])
for algo11_Colt in res_11_Colt.history:
    opt11_Colt = algo11_Colt.opt
    feas11_Colt = np.where(opt11_Colt.get("feasible"))[0]
    hist_F11_Colt.append(opt11_Colt.get("F")[feas11_Colt])
for algo11_Funet in res_11_Funet.history:
    opt11_Funet = algo11_Funet.opt
    feas11_Funet = np.where(opt11_Funet.get("feasible"))[0]
    hist_F11_Funet.append(opt11_Funet.get("F")[feas11_Funet])
for algo11_Abvt in res_11_Abvt.history:
    opt11_Abvt = algo11_Abvt.opt
    feas11_Abvt = np.where(opt11_Abvt.get("feasible"))[0]
    hist_F11_Abvt.append(opt11_Abvt.get("F")[feas11_Abvt])
for algo11_Intellifiber in res_11_Intellifiber.history:
    opt11_Intellifiber = algo11_Intellifiber.opt
    feas11_Intellifiber = np.where(opt11_Intellifiber.get("feasible"))[0]
    hist_F11_Intellifiber.append(opt11_Intellifiber.get("F")[feas11_Intellifiber])
for algo11_TataNld in res_11_TataNld.history:
    opt11_TataNld = algo11_TataNld.opt
    feas11_TataNld = np.where(opt11_TataNld.get("feasible"))[0]
    hist_F11_TataNld.append(opt11_TataNld.get("F")[feas11_TataNld])
for algo11_Internode in res_11_Internode.history:
    opt11_Internode = algo11_Internode.opt
    feas11_Internode= np.where(opt11_Internode.get("feasible"))[0]
    hist_F11_Internode.append(opt11_Internode.get("F")[feas11_Internode])
for algo11_Missouri in res_11_Missouri.history:
    opt11_Missouri = algo11_Missouri.opt
    feas11_Missouri = np.where(opt11_Missouri.get("feasible"))[0]
    hist_F11_Missouri.append(opt11_Missouri.get("F")[feas11_Missouri])
for algo11_Ion in res_11_Ion.history:
    opt11_Ion = algo11_Ion.opt
    feas11_Ion = np.where(opt11_Ion.get("feasible"))[0]
    hist_F11_Ion.append(opt11_Ion.get("F")[feas11_Ion])
for algo11_Palmetto in res_11_Palmetto.history:
    opt11_Palmetto = algo11_Palmetto.opt
    feas11_Palmetto = np.where(opt11_Palmetto.get("feasible"))[0]
    hist_F11_Palmetto.append(opt11_Palmetto.get("F")[feas11_Palmetto])

for algo12_Cogent in res_12_Cogent.history:
    opt12_Cogent = algo12_Cogent.opt
    feas12_Cogent = np.where(opt12_Cogent.get("feasible"))[0]
    hist_F12_Cogent.append(opt12_Cogent.get("F")[feas12_Cogent])
for algo12_UsCarrier in res_12_UsCarrier.history:
    opt12_UsCarrier = algo12_UsCarrier.opt
    feas12_UsCarrier = np.where(opt12_UsCarrier.get("feasible"))[0]
    hist_F12_UsCarrier.append(opt12_UsCarrier.get("F")[feas12_UsCarrier])
for algo12_HiberniaGlobal in res_12_HiberniaGlobal.history:
    opt12_HiberniaGlobal = algo12_HiberniaGlobal.opt
    feas12_HiberniaGlobal = np.where(opt12_HiberniaGlobal.get("feasible"))[0]
    hist_F12_HiberniaGlobal.append(opt12_HiberniaGlobal.get("F")[feas12_HiberniaGlobal])
for algo12_Colt in res_12_Colt.history:
    opt12_Colt = algo12_Colt.opt
    feas12_Colt = np.where(opt12_Colt.get("feasible"))[0]
    hist_F12_Colt.append(opt12_Colt.get("F")[feas12_Colt])
for algo12_Funet in res_12_Funet.history:
    opt12_Funet = algo12_Funet.opt
    feas12_Funet = np.where(opt12_Funet.get("feasible"))[0]
    hist_F12_Funet.append(opt12_Funet.get("F")[feas12_Funet])
for algo12_Abvt in res_12_Abvt.history:
    opt12_Abvt = algo12_Abvt.opt
    feas12_Abvt = np.where(opt12_Abvt.get("feasible"))[0]
    hist_F12_Abvt.append(opt12_Abvt.get("F")[feas12_Abvt])
for algo12_Intellifiber in res_12_Intellifiber.history:
    opt12_Intellifiber = algo12_Intellifiber.opt
    feas12_Intellifiber = np.where(opt12_Intellifiber.get("feasible"))[0]
    hist_F12_Intellifiber.append(opt12_Intellifiber.get("F")[feas12_Intellifiber])
for algo12_TataNld in res_12_TataNld.history:
    opt12_TataNld = algo12_TataNld.opt
    feas12_TataNld = np.where(opt12_TataNld.get("feasible"))[0]
    hist_F12_TataNld.append(opt12_TataNld.get("F")[feas12_TataNld])
for algo12_Internode in res_12_Internode.history:
    opt12_Internode = algo12_Internode.opt
    feas12_Internode= np.where(opt12_Internode.get("feasible"))[0]
    hist_F12_Internode.append(opt12_Internode.get("F")[feas12_Internode])
for algo12_Missouri in res_12_Missouri.history:
    opt12_Missouri = algo12_Missouri.opt
    feas12_Missouri = np.where(opt12_Missouri.get("feasible"))[0]
    hist_F12_Missouri.append(opt12_Missouri.get("F")[feas12_Missouri])
for algo12_Ion in res_12_Ion.history:
    opt12_Ion = algo12_Ion.opt
    feas12_Ion = np.where(opt12_Ion.get("feasible"))[0]
    hist_F12_Ion.append(opt12_Ion.get("F")[feas12_Ion])
for algo12_Palmetto in res_12_Palmetto.history:
    opt12_Palmetto = algo12_Palmetto.opt
    feas12_Palmetto = np.where(opt12_Palmetto.get("feasible"))[0]
    hist_F12_Palmetto.append(opt12_Palmetto.get("F")[feas12_Palmetto])

for algo13_Cogent in res_13_Cogent.history:
    opt13_Cogent = algo13_Cogent.opt
    feas13_Cogent = np.where(opt13_Cogent.get("feasible"))[0]
    hist_F13_Cogent.append(opt13_Cogent.get("F")[feas13_Cogent])
for algo13_UsCarrier in res_13_UsCarrier.history:
    opt13_UsCarrier = algo13_UsCarrier.opt
    feas13_UsCarrier = np.where(opt13_UsCarrier.get("feasible"))[0]
    hist_F13_UsCarrier.append(opt13_UsCarrier.get("F")[feas13_UsCarrier])
for algo13_HiberniaGlobal in res_13_HiberniaGlobal.history:
    opt13_HiberniaGlobal = algo13_HiberniaGlobal.opt
    feas13_HiberniaGlobal = np.where(opt13_HiberniaGlobal.get("feasible"))[0]
    hist_F13_HiberniaGlobal.append(opt13_HiberniaGlobal.get("F")[feas13_HiberniaGlobal])
for algo13_Colt in res_13_Colt.history:
    opt13_Colt = algo13_Colt.opt
    feas13_Colt = np.where(opt13_Colt.get("feasible"))[0]
    hist_F13_Colt.append(opt13_Colt.get("F")[feas13_Colt])
for algo13_Funet in res_13_Funet.history:
    opt13_Funet = algo13_Funet.opt
    feas13_Funet = np.where(opt13_Funet.get("feasible"))[0]
    hist_F13_Funet.append(opt13_Funet.get("F")[feas13_Funet])
for algo13_Abvt in res_13_Abvt.history:
    opt13_Abvt = algo13_Abvt.opt
    feas13_Abvt = np.where(opt13_Abvt.get("feasible"))[0]
    hist_F13_Abvt.append(opt13_Abvt.get("F")[feas13_Abvt])
for algo13_Intellifiber in res_13_Intellifiber.history:
    opt13_Intellifiber = algo13_Intellifiber.opt
    feas13_Intellifiber = np.where(opt13_Intellifiber.get("feasible"))[0]
    hist_F13_Intellifiber.append(opt13_Intellifiber.get("F")[feas13_Intellifiber])
for algo13_TataNld in res_13_TataNld.history:
    opt13_TataNld = algo13_TataNld.opt
    feas13_TataNld = np.where(opt13_TataNld.get("feasible"))[0]
    hist_F13_TataNld.append(opt13_TataNld.get("F")[feas13_TataNld])
for algo13_Internode in res_13_Internode.history:
    opt13_Internode = algo13_Internode.opt
    feas13_Internode= np.where(opt13_Internode.get("feasible"))[0]
    hist_F13_Internode.append(opt13_Internode.get("F")[feas13_Internode])
for algo13_Missouri in res_13_Missouri.history:
    opt13_Missouri = algo13_Missouri.opt
    feas13_Missouri = np.where(opt13_Missouri.get("feasible"))[0]
    hist_F13_Missouri.append(opt13_Missouri.get("F")[feas13_Missouri])
for algo13_Ion in res_13_Ion.history:
    opt13_Ion = algo13_Ion.opt
    feas13_Ion = np.where(opt13_Ion.get("feasible"))[0]
    hist_F13_Ion.append(opt13_Ion.get("F")[feas13_Ion])
for algo13_Palmetto in res_13_Palmetto.history:
    opt13_Palmetto = algo13_Palmetto.opt
    feas13_Palmetto = np.where(opt13_Palmetto.get("feasible"))[0]
    hist_F13_Palmetto.append(opt13_Palmetto.get("F")[feas13_Palmetto])

2. Calculate hypervolume and merge them (Mean value) for each algorithm

In [ ]:
from pymoo.indicators.hv import Hypervolume

metric = Hypervolume(ref_point= np.array(ref_point))

hv_1_Cogent = [metric.do(_F1_Cogent) for _F1_Cogent in hist_F1_Cogent]
hv_1_UsCarrier = [metric.do(_F1_UsCarrier) for _F1_UsCarrier in hist_F1_UsCarrier]
hv_1_HiberniaGlobal = [metric.do(_F1_HiberniaGlobal) for _F1_HiberniaGlobal in hist_F1_HiberniaGlobal]
hv_1_Colt = [metric.do(_F1_Colt) for _F1_Colt in hist_F1_Colt]
hv_1_Funet = [metric.do(_F1_Funet) for _F1_Funet in hist_F1_Funet]
hv_1_Abvt = [metric.do(_F1_Abvt) for _F1_Abvt in hist_F1_Abvt]
hv_1_Intellifiber = [metric.do(_F1_Intellifiber) for _F1_Intellifiber in hist_F1_Intellifiber]
hv_1_TataNld = [metric.do(_F1_TataNld) for _F1_TataNld in hist_F1_TataNld]
hv_1_Internode = [metric.do(_F1_Internode) for _F1_Internode in hist_F1_Internode]
hv_1_Missouri = [metric.do(_F1_Missouri) for _F1_Missouri in hist_F1_Missouri]
hv_1_Ion = [metric.do(_F1_Ion) for _F1_Ion in hist_F1_Ion]
hv_1_Palmetto = [metric.do(_F1_Palmetto) for _F1_Palmetto in hist_F1_Palmetto]

hv_2_Cogent = [metric.do(_F2_Cogent) for _F2_Cogent in hist_F2_Cogent]
hv_2_UsCarrier = [metric.do(_F2_UsCarrier) for _F2_UsCarrier in hist_F2_UsCarrier]
hv_2_HiberniaGlobal = [metric.do(_F2_HiberniaGlobal) for _F2_HiberniaGlobal in hist_F2_HiberniaGlobal]
hv_2_Colt = [metric.do(_F2_Colt) for _F2_Colt in hist_F2_Colt]
hv_2_Funet = [metric.do(_F2_Funet) for _F2_Funet in hist_F2_Funet]
hv_2_Abvt = [metric.do(_F2_Abvt) for _F2_Abvt in hist_F2_Abvt]
hv_2_Intellifiber = [metric.do(_F2_Intellifiber) for _F2_Intellifiber in hist_F2_Intellifiber]
hv_2_TataNld = [metric.do(_F2_TataNld) for _F2_TataNld in hist_F2_TataNld]
hv_2_Internode = [metric.do(_F2_Internode) for _F2_Internode in hist_F2_Internode]
hv_2_Missouri = [metric.do(_F2_Missouri) for _F2_Missouri in hist_F2_Missouri]
hv_2_Ion = [metric.do(_F2_Ion) for _F2_Ion in hist_F2_Ion]
hv_2_Palmetto = [metric.do(_F2_Palmetto) for _F2_Palmetto in hist_F2_Palmetto]

hv_3_Cogent = [metric.do(_F3_Cogent) for _F3_Cogent in hist_F3_Cogent]
hv_3_UsCarrier = [metric.do(_F3_UsCarrier) for _F3_UsCarrier in hist_F3_UsCarrier]
hv_3_HiberniaGlobal = [metric.do(_F3_HiberniaGlobal) for _F3_HiberniaGlobal in hist_F3_HiberniaGlobal]
hv_3_Colt = [metric.do(_F3_Colt) for _F3_Colt in hist_F3_Colt]
hv_3_Funet = [metric.do(_F3_Funet) for _F3_Funet in hist_F3_Funet]
hv_3_Abvt = [metric.do(_F3_Abvt) for _F3_Abvt in hist_F3_Abvt]
hv_3_Intellifiber = [metric.do(_F3_Intellifiber) for _F3_Intellifiber in hist_F3_Intellifiber]
hv_3_TataNld = [metric.do(_F3_TataNld) for _F3_TataNld in hist_F3_TataNld]
hv_3_Internode = [metric.do(_F3_Internode) for _F3_Internode in hist_F3_Internode]
hv_3_Missouri = [metric.do(_F3_Missouri) for _F3_Missouri in hist_F3_Missouri]
hv_3_Ion = [metric.do(_F3_Ion) for _F3_Ion in hist_F3_Ion]
hv_3_Palmetto = [metric.do(_F3_Palmetto) for _F3_Palmetto in hist_F3_Palmetto]

hv_4_Cogent = [metric.do(_F4_Cogent) for _F4_Cogent in hist_F4_Cogent]
hv_4_UsCarrier = [metric.do(_F4_UsCarrier) for _F4_UsCarrier in hist_F4_UsCarrier]
hv_4_HiberniaGlobal = [metric.do(_F4_HiberniaGlobal) for _F4_HiberniaGlobal in hist_F4_HiberniaGlobal]
hv_4_Colt = [metric.do(_F4_Colt) for _F4_Colt in hist_F4_Colt]
hv_4_Funet = [metric.do(_F4_Funet) for _F4_Funet in hist_F4_Funet]
hv_4_Abvt = [metric.do(_F4_Abvt) for _F4_Abvt in hist_F4_Abvt]
hv_4_Intellifiber = [metric.do(_F4_Intellifiber) for _F4_Intellifiber in hist_F4_Intellifiber]
hv_4_TataNld = [metric.do(_F4_TataNld) for _F4_TataNld in hist_F4_TataNld]
hv_4_Internode = [metric.do(_F4_Internode) for _F4_Internode in hist_F4_Internode]
hv_4_Missouri = [metric.do(_F4_Missouri) for _F4_Missouri in hist_F4_Missouri]
hv_4_Ion = [metric.do(_F4_Ion) for _F4_Ion in hist_F4_Ion]
hv_4_Palmetto = [metric.do(_F4_Palmetto) for _F4_Palmetto in hist_F4_Palmetto]

hv_5_Cogent = [metric.do(_F5_Cogent) for _F5_Cogent in hist_F5_Cogent]
hv_5_UsCarrier = [metric.do(_F5_UsCarrier) for _F5_UsCarrier in hist_F5_UsCarrier]
hv_5_HiberniaGlobal = [metric.do(_F5_HiberniaGlobal) for _F5_HiberniaGlobal in hist_F5_HiberniaGlobal]
hv_5_Colt = [metric.do(_F5_Colt) for _F5_Colt in hist_F5_Colt]
hv_5_Funet = [metric.do(_F5_Funet) for _F5_Funet in hist_F5_Funet]
hv_5_Abvt = [metric.do(_F5_Abvt) for _F5_Abvt in hist_F5_Abvt]
hv_5_Intellifiber = [metric.do(_F5_Intellifiber) for _F5_Intellifiber in hist_F5_Intellifiber]
hv_5_TataNld = [metric.do(_F5_TataNld) for _F5_TataNld in hist_F5_TataNld]
hv_5_Internode = [metric.do(_F5_Internode) for _F5_Internode in hist_F5_Internode]
hv_5_Missouri = [metric.do(_F5_Missouri) for _F5_Missouri in hist_F5_Missouri]
hv_5_Ion = [metric.do(_F5_Ion) for _F5_Ion in hist_F5_Ion]
hv_5_Palmetto = [metric.do(_F5_Palmetto) for _F5_Palmetto in hist_F5_Palmetto]

hv_6_Cogent = [metric.do(_F6_Cogent) for _F6_Cogent in hist_F6_Cogent]
hv_6_UsCarrier = [metric.do(_F6_UsCarrier) for _F6_UsCarrier in hist_F6_UsCarrier]
hv_6_HiberniaGlobal = [metric.do(_F6_HiberniaGlobal) for _F6_HiberniaGlobal in hist_F6_HiberniaGlobal]
hv_6_Colt = [metric.do(_F6_Colt) for _F6_Colt in hist_F6_Colt]
hv_6_Funet = [metric.do(_F6_Funet) for _F6_Funet in hist_F6_Funet]
hv_6_Abvt = [metric.do(_F6_Abvt) for _F6_Abvt in hist_F6_Abvt]
hv_6_Intellifiber = [metric.do(_F6_Intellifiber) for _F6_Intellifiber in hist_F6_Intellifiber]
hv_6_TataNld = [metric.do(_F6_TataNld) for _F6_TataNld in hist_F6_TataNld]
hv_6_Internode = [metric.do(_F6_Internode) for _F6_Internode in hist_F6_Internode]
hv_6_Missouri = [metric.do(_F6_Missouri) for _F6_Missouri in hist_F6_Missouri]
hv_6_Ion = [metric.do(_F6_Ion) for _F6_Ion in hist_F6_Ion]
hv_6_Palmetto = [metric.do(_F6_Palmetto) for _F6_Palmetto in hist_F6_Palmetto]

hv_7_Cogent = [metric.do(_F7_Cogent) for _F7_Cogent in hist_F7_Cogent]
hv_7_UsCarrier = [metric.do(_F7_UsCarrier) for _F7_UsCarrier in hist_F7_UsCarrier]
hv_7_HiberniaGlobal = [metric.do(_F7_HiberniaGlobal) for _F7_HiberniaGlobal in hist_F7_HiberniaGlobal]
hv_7_Colt = [metric.do(_F7_Colt) for _F7_Colt in hist_F7_Colt]
hv_7_Funet = [metric.do(_F7_Funet) for _F7_Funet in hist_F7_Funet]
hv_7_Abvt = [metric.do(_F7_Abvt) for _F7_Abvt in hist_F7_Abvt]
hv_7_Intellifiber = [metric.do(_F7_Intellifiber) for _F7_Intellifiber in hist_F7_Intellifiber]
hv_7_TataNld = [metric.do(_F7_TataNld) for _F7_TataNld in hist_F7_TataNld]
hv_7_Internode = [metric.do(_F7_Internode) for _F7_Internode in hist_F7_Internode]
hv_7_Missouri = [metric.do(_F7_Missouri) for _F7_Missouri in hist_F7_Missouri]
hv_7_Ion = [metric.do(_F7_Ion) for _F7_Ion in hist_F7_Ion]
hv_7_Palmetto = [metric.do(_F7_Palmetto) for _F7_Palmetto in hist_F7_Palmetto]

hv_8_Cogent = [metric.do(_F8_Cogent) for _F8_Cogent in hist_F8_Cogent]
hv_8_UsCarrier = [metric.do(_F8_UsCarrier) for _F8_UsCarrier in hist_F8_UsCarrier]
hv_8_HiberniaGlobal = [metric.do(_F8_HiberniaGlobal) for _F8_HiberniaGlobal in hist_F8_HiberniaGlobal]
hv_8_Colt = [metric.do(_F8_Colt) for _F8_Colt in hist_F8_Colt]
hv_8_Funet = [metric.do(_F8_Funet) for _F8_Funet in hist_F8_Funet]
hv_8_Abvt = [metric.do(_F8_Abvt) for _F8_Abvt in hist_F8_Abvt]
hv_8_Intellifiber = [metric.do(_F8_Intellifiber) for _F8_Intellifiber in hist_F8_Intellifiber]
hv_8_TataNld = [metric.do(_F8_TataNld) for _F8_TataNld in hist_F8_TataNld]
hv_8_Internode = [metric.do(_F8_Internode) for _F8_Internode in hist_F8_Internode]
hv_8_Missouri = [metric.do(_F8_Missouri) for _F8_Missouri in hist_F8_Missouri]
hv_8_Ion = [metric.do(_F8_Ion) for _F8_Ion in hist_F8_Ion]
hv_8_Palmetto = [metric.do(_F8_Palmetto) for _F8_Palmetto in hist_F8_Palmetto]

hv_9_Cogent = [metric.do(_F9_Cogent) for _F9_Cogent in hist_F9_Cogent]
hv_9_UsCarrier = [metric.do(_F9_UsCarrier) for _F9_UsCarrier in hist_F9_UsCarrier]
hv_9_HiberniaGlobal = [metric.do(_F9_HiberniaGlobal) for _F9_HiberniaGlobal in hist_F9_HiberniaGlobal]
hv_9_Colt = [metric.do(_F9_Colt) for _F9_Colt in hist_F9_Colt]
hv_9_Funet = [metric.do(_F9_Funet) for _F9_Funet in hist_F9_Funet]
hv_9_Abvt = [metric.do(_F9_Abvt) for _F9_Abvt in hist_F9_Abvt]
hv_9_Intellifiber = [metric.do(_F9_Intellifiber) for _F9_Intellifiber in hist_F9_Intellifiber]
hv_9_TataNld = [metric.do(_F9_TataNld) for _F9_TataNld in hist_F9_TataNld]
hv_9_Internode = [metric.do(_F9_Internode) for _F9_Internode in hist_F9_Internode]
hv_9_Missouri = [metric.do(_F9_Missouri) for _F9_Missouri in hist_F9_Missouri]
hv_9_Ion = [metric.do(_F9_Ion) for _F9_Ion in hist_F9_Ion]
hv_9_Palmetto = [metric.do(_F9_Palmetto) for _F9_Palmetto in hist_F9_Palmetto]

hv_10_Cogent = [metric.do(_F10_Cogent) for _F10_Cogent in hist_F10_Cogent]
hv_10_UsCarrier = [metric.do(_F10_UsCarrier) for _F10_UsCarrier in hist_F10_UsCarrier]
hv_10_HiberniaGlobal = [metric.do(_F10_HiberniaGlobal) for _F10_HiberniaGlobal in hist_F10_HiberniaGlobal]
hv_10_Colt = [metric.do(_F10_Colt) for _F10_Colt in hist_F10_Colt]
hv_10_Funet = [metric.do(_F10_Funet) for _F10_Funet in hist_F10_Funet]
hv_10_Abvt = [metric.do(_F10_Abvt) for _F10_Abvt in hist_F10_Abvt]
hv_10_Intellifiber = [metric.do(_F10_Intellifiber) for _F10_Intellifiber in hist_F10_Intellifiber]
hv_10_TataNld = [metric.do(_F10_TataNld) for _F10_TataNld in hist_F10_TataNld]
hv_10_Internode = [metric.do(_F10_Internode) for _F10_Internode in hist_F10_Internode]
hv_10_Missouri = [metric.do(_F10_Missouri) for _F10_Missouri in hist_F10_Missouri]
hv_10_Ion = [metric.do(_F10_Ion) for _F10_Ion in hist_F10_Ion]
hv_10_Palmetto = [metric.do(_F10_Palmetto) for _F10_Palmetto in hist_F10_Palmetto]

hv_11_Cogent = [metric.do(_F11_Cogent) for _F11_Cogent in hist_F11_Cogent]
hv_11_UsCarrier = [metric.do(_F11_UsCarrier) for _F11_UsCarrier in hist_F11_UsCarrier]
hv_11_HiberniaGlobal = [metric.do(_F11_HiberniaGlobal) for _F11_HiberniaGlobal in hist_F11_HiberniaGlobal]
hv_11_Colt = [metric.do(_F11_Colt) for _F11_Colt in hist_F11_Colt]
hv_11_Funet = [metric.do(_F11_Funet) for _F11_Funet in hist_F11_Funet]
hv_11_Abvt = [metric.do(_F11_Abvt) for _F11_Abvt in hist_F11_Abvt]
hv_11_Intellifiber = [metric.do(_F11_Intellifiber) for _F11_Intellifiber in hist_F11_Intellifiber]
hv_11_TataNld = [metric.do(_F11_TataNld) for _F11_TataNld in hist_F11_TataNld]
hv_11_Internode = [metric.do(_F11_Internode) for _F11_Internode in hist_F11_Internode]
hv_11_Missouri = [metric.do(_F11_Missouri) for _F11_Missouri in hist_F11_Missouri]
hv_11_Ion = [metric.do(_F11_Ion) for _F11_Ion in hist_F11_Ion]
hv_11_Palmetto = [metric.do(_F11_Palmetto) for _F11_Palmetto in hist_F11_Palmetto]

hv_12_Cogent = [metric.do(_F12_Cogent) for _F12_Cogent in hist_F12_Cogent]
hv_12_UsCarrier = [metric.do(_F12_UsCarrier) for _F12_UsCarrier in hist_F12_UsCarrier]
hv_12_HiberniaGlobal = [metric.do(_F12_HiberniaGlobal) for _F12_HiberniaGlobal in hist_F12_HiberniaGlobal]
hv_12_Colt = [metric.do(_F12_Colt) for _F12_Colt in hist_F12_Colt]
hv_12_Funet = [metric.do(_F12_Funet) for _F12_Funet in hist_F12_Funet]
hv_12_Abvt = [metric.do(_F12_Abvt) for _F12_Abvt in hist_F12_Abvt]
hv_12_Intellifiber = [metric.do(_F12_Intellifiber) for _F12_Intellifiber in hist_F12_Intellifiber]
hv_12_TataNld = [metric.do(_F12_TataNld) for _F12_TataNld in hist_F12_TataNld]
hv_12_Internode = [metric.do(_F12_Internode) for _F12_Internode in hist_F12_Internode]
hv_12_Missouri = [metric.do(_F12_Missouri) for _F12_Missouri in hist_F12_Missouri]
hv_12_Ion = [metric.do(_F12_Ion) for _F12_Ion in hist_F12_Ion]
hv_12_Palmetto = [metric.do(_F12_Palmetto) for _F12_Palmetto in hist_F12_Palmetto]

hv_13_Cogent = [metric.do(_F13_Cogent) for _F13_Cogent in hist_F13_Cogent]
hv_13_UsCarrier = [metric.do(_F13_UsCarrier) for _F13_UsCarrier in hist_F13_UsCarrier]
hv_13_HiberniaGlobal = [metric.do(_F13_HiberniaGlobal) for _F13_HiberniaGlobal in hist_F13_HiberniaGlobal]
hv_13_Colt = [metric.do(_F13_Colt) for _F13_Colt in hist_F13_Colt]
hv_13_Funet = [metric.do(_F13_Funet) for _F13_Funet in hist_F13_Funet]
hv_13_Abvt = [metric.do(_F13_Abvt) for _F13_Abvt in hist_F13_Abvt]
hv_13_Intellifiber = [metric.do(_F13_Intellifiber) for _F13_Intellifiber in hist_F13_Intellifiber]
hv_13_TataNld = [metric.do(_F13_TataNld) for _F13_TataNld in hist_F13_TataNld]
hv_13_Internode = [metric.do(_F13_Internode) for _F13_Internode in hist_F13_Internode]
hv_13_Missouri = [metric.do(_F13_Missouri) for _F13_Missouri in hist_F13_Missouri]
hv_13_Ion = [metric.do(_F13_Ion) for _F13_Ion in hist_F13_Ion]
hv_13_Palmetto = [metric.do(_F13_Palmetto) for _F13_Palmetto in hist_F13_Palmetto]

# agemoea
hv_1 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_1_Cogent, hv_1_UsCarrier, hv_1_HiberniaGlobal, hv_1_Colt, hv_1_Funet, hv_1_Abvt, hv_1_Intellifiber, hv_1_TataNld, hv_1_Internode, hv_1_Missouri, hv_1_Ion, hv_1_Palmetto)]

# agemoea2
hv_2 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_2_Cogent, hv_2_UsCarrier, hv_2_HiberniaGlobal, hv_2_Colt, hv_2_Funet, hv_2_Abvt, hv_2_Intellifiber, hv_2_TataNld, hv_2_Internode, hv_2_Missouri, hv_2_Ion, hv_2_Palmetto)]

# nsga2
hv_3 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_3_Cogent, hv_3_UsCarrier, hv_3_HiberniaGlobal, hv_3_Colt, hv_3_Funet, hv_3_Abvt, hv_3_Intellifiber, hv_3_TataNld, hv_3_Internode, hv_3_Missouri, hv_3_Ion, hv_3_Palmetto)]

# nsga3
hv_4 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_4_Cogent, hv_4_UsCarrier, hv_4_HiberniaGlobal, hv_4_Colt, hv_4_Funet, hv_4_Abvt, hv_4_Intellifiber, hv_4_TataNld, hv_4_Internode, hv_4_Missouri, hv_4_Ion, hv_4_Palmetto)]

# rnsga2
hv_5 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_5_Cogent, hv_5_UsCarrier, hv_5_HiberniaGlobal, hv_5_Colt, hv_5_Funet, hv_5_Abvt, hv_5_Intellifiber, hv_5_TataNld, hv_5_Internode, hv_5_Missouri, hv_5_Ion, hv_5_Palmetto)]

# rnsga3
hv_6 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_6_Cogent, hv_6_UsCarrier, hv_6_HiberniaGlobal, hv_6_Colt, hv_6_Funet, hv_6_Abvt, hv_6_Intellifiber, hv_6_TataNld, hv_6_Internode, hv_6_Missouri, hv_6_Ion, hv_6_Palmetto)]

# rvea
hv_7 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_7_Cogent, hv_7_UsCarrier, hv_7_HiberniaGlobal, hv_7_Colt, hv_7_Funet, hv_7_Abvt, hv_7_Intellifiber, hv_7_TataNld, hv_7_Internode, hv_7_Missouri, hv_7_Ion, hv_7_Palmetto)]

# smsemoa
hv_8 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_8_Cogent, hv_8_UsCarrier, hv_8_HiberniaGlobal, hv_8_Colt, hv_8_Funet, hv_8_Abvt, hv_8_Intellifiber, hv_8_TataNld, hv_8_Internode, hv_8_Missouri, hv_8_Ion, hv_8_Palmetto)]

# unsga3
hv_9 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_9_Cogent, hv_9_UsCarrier, hv_9_HiberniaGlobal, hv_9_Colt, hv_9_Funet, hv_9_Abvt, hv_9_Intellifiber, hv_9_TataNld, hv_9_Internode, hv_9_Missouri, hv_9_Ion, hv_9_Palmetto)]

# ctaea
hv_10 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_10_Cogent, hv_10_UsCarrier, hv_10_HiberniaGlobal, hv_10_Colt, hv_10_Funet, hv_10_Abvt, hv_10_Intellifiber, hv_10_TataNld, hv_10_Internode, hv_10_Missouri, hv_10_Ion, hv_10_Palmetto)]

# dnsga2
hv_11 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_11_Cogent, hv_11_UsCarrier, hv_11_HiberniaGlobal, hv_11_Colt, hv_11_Funet, hv_11_Abvt, hv_11_Intellifiber, hv_11_TataNld, hv_11_Internode, hv_11_Missouri, hv_11_Ion, hv_11_Palmetto)]

# kgbdmoea
hv_12 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_12_Cogent, hv_12_UsCarrier, hv_12_HiberniaGlobal, hv_12_Colt, hv_12_Funet, hv_12_Abvt, hv_12_Intellifiber, hv_12_TataNld, hv_12_Internode, hv_12_Missouri, hv_12_Ion, hv_12_Palmetto)]

# moead
hv_13 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_13_Cogent, hv_13_UsCarrier, hv_13_HiberniaGlobal, hv_13_Colt, hv_13_Funet, hv_13_Abvt, hv_13_Intellifiber, hv_13_TataNld, hv_13_Internode, hv_13_Missouri, hv_13_Ion, hv_13_Palmetto)]

In [ ]:
print('agemoea:')
print(hv_1)
print('agemoea2:')
print(hv_2)
print('nsga2:')
print(hv_3)
print('nsga3:')
print(hv_4)
print('rnsga2:')
print(hv_5)
print('rnsga3:')
print(hv_6)
print('rvea:')
print(hv_7)
print('smsemoa:')
print(hv_8)
print('unsga3:')
print(hv_9)
print('ctaea:')
print(hv_10)
print('dnsga2:')
print(hv_11)
print('kgbdmoea:')
print(hv_12)
print('moead:')
print(hv_13)

3. Draw mean hypervolume for each algorithm

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations") # brown,pink,#5733FF,olive,cyan,#FF5733,#33FF57
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

## Draw AGEMOEA

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

## Draw AGEMOEA2

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

## Draw NSGA2

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

## Draw NSGA3

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

## Draw RNSGA2

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

## Draw RNSGA3

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

## Draw RVEA

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

## Draw SMSEMOA

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

## Draw UNSGA3

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

## Draw CTAEA

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

## Draw DNSGA2

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

## Draw KGBDMOEA

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

## Draw MOEAD

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()